In [ ]:
# Testování dotazů do KN podle [katastrální území] [parc.č.]
#/////////////////////////////////////////////////////////////////////////////////////////
#/////////////////////////////////////////////////////////////////////////////////////////

import requests
from lxml import etree
from pyproj import Transformer
import pandas as pd
import os

# ==============================================================================
# 1. Funkce pro získání kódu katastrálního území (UPPER_ZONING_ID) podle názvu
# ==============================================================================
def get_zoning_code(zoning_name):
    """
    Na základě názvu katastrálního území získá identifikační kód (UPPER_ZONING_ID)
    pomocí uloženého dotazu GetZoningByName.
    """
    base_url = "https://services.cuzk.cz/wfs/inspire-CP-wfs.asp?"
    params = {
        "service": "WFS",
        "version": "2.0.0",
        "request": "GetFeature",
        "storedQuery_id": "GetZoningByName",
        "ZONING_NAME": zoning_name
    }
    print(f"\nDotazuji katastrální území (GetZoningByName) pro: {zoning_name}")
    response = requests.get(base_url, params=params)
    print("HTTP status kód (GetZoningByName):", response.status_code)
    
    # Uložení odpovědi pro případnou analýzu
    filename = "zoning_response.xml"
    with open(filename, "wb") as f:
        f.write(response.content)
    print(f"XML odpověď (GetZoningByName) byla uložena do souboru: {os.path.abspath(filename)}")
    
    try: 
        tree = etree.fromstring(response.content)
    except Exception as e:
        raise Exception(f"Chyba při parsování XML odpovědi (GetZoningByName): {e}")
    
    ns = {
        "CP": "http://inspire.ec.europa.eu/schemas/cp/4.0",
        "base": "http://inspire.ec.europa.eu/schemas/base/3.3",
        "wfs": "http://www.opengis.net/wfs/2.0"
    }
    # Vyhledání identifikátoru katastrálního území v elementu CP:CadastralZoning/CP:inspireId/base:Identifier/base:localId
    zoning_id_elem = tree.find(".//CP:CadastralZoning/CP:inspireId/base:Identifier/base:localId", namespaces=ns)
    if zoning_id_elem is not None and zoning_id_elem.text:
        zoning_id_full = zoning_id_elem.text.strip()
        print("Nalezený identifikátor (full):", zoning_id_full)
        zoning_id = zoning_id_full[3:] if zoning_id_full.startswith("CZ.") else zoning_id_full
        print("Vyhodnocený kód katastrálního území (UPPER_ZONING_ID):", zoning_id)
        return zoning_id
    else:
        print("Nebyl nalezen identifikátor katastrálního území v odpovědi.")
        return None

# ==============================================================================
# 2. Funkce pro získání dat o parcele a vytvoření DataFrame
# ==============================================================================
def get_parcel_data(zoning_name, parcel_number):
    """
    Na základě názvu katastrálního území a parcelního čísla získá podrobné informace o parcele.
    Nejprve získá kód katastrálního území, poté zavolá storedQuery GetParcel.
    Z odpovědi extrahuje informace a uloží je do pandas DataFrame vhodného pro import do DB.
    """
    # Získání kódu katastrálního území
    upper_zoning_id = get_zoning_code(zoning_name)
    if not upper_zoning_id:
        raise Exception("Nepodařilo se získat kód katastrálního území.")
    
    base_url = "https://services.cuzk.cz/wfs/inspire-CP-wfs.asp?"
    params = {
        "service": "WFS",
        "version": "2.0.0",
        "request": "GetFeature",
        "storedQuery_id": "GetParcel",
        "UPPER_ZONING_ID": upper_zoning_id,
        "TEXT": parcel_number
    }
    
    print("\nSestavuji dotaz pomocí storedQuery 'GetParcel'...")
    print("Základní URL:", base_url)
    print("Parametry:")
    for k, v in params.items():
        print(f"  {k}: {v}")
    
    response = requests.get(base_url, params=params)
    print("\nHTTP status kód odpovědi (GetParcel):", response.status_code)
    
    filename = "parcel_response.xml"
    with open(filename, "wb") as f:
        f.write(response.content)
    print(f"XML odpověď (GetParcel) byla uložena do souboru: {os.path.abspath(filename)}")
    
    try:
        tree = etree.fromstring(response.content)
    except Exception as e:
        raise Exception(f"Chyba při parsování XML odpovědi (GetParcel): {e}")
    
    fc = tree.xpath("/wfs:FeatureCollection", namespaces={"wfs": "http://www.opengis.net/wfs/2.0"})
    if fc:
        fc_elem = fc[0]
        num_matched = fc_elem.get("numberMatched", "není uvedeno")
        num_returned = fc_elem.get("numberReturned", "není uvedeno")
        print("\nFeatureCollection:")
        print("  numberMatched =", num_matched)
        print("  numberReturned =", num_returned)
    else:
        print("Element FeatureCollection nebyl nalezen.")
    
    # Rozšířený slovník jmenných prostorů – zahrnujeme i "base"
    ns = {
        "wfs": "http://www.opengis.net/wfs/2.0",
        "gml": "http://www.opengis.net/gml/3.2",
        "CP": "http://inspire.ec.europa.eu/schemas/cp/4.0",
        "base": "http://inspire.ec.europa.eu/schemas/base/3.3"
    }
    
    members = tree.xpath("//wfs:member", namespaces=ns)
    print(f"\nNalezeno {len(members)} feature (member).")
    
    records = []
    
    for mem in members:
        parcel_elem = mem.find(".//CP:CadastralParcel", namespaces=ns)
        if parcel_elem is None:
            continue
        
        # gml:id z atributu CP:CadastralParcel
        gml_id = parcel_elem.get("{http://www.opengis.net/gml/3.2}id")
        
        # cp:areaValue
        area_elem = parcel_elem.find("CP:areaValue", namespaces=ns)
        try:
            area_val = float(area_elem.text.strip()) if area_elem is not None and area_elem.text else None
        except ValueError:
            area_val = None
        
        # cp:beginLifespanVersion
        begin_elem = parcel_elem.find("CP:beginLifespanVersion", namespaces=ns)
        begin_val = begin_elem.text.strip() if begin_elem is not None and begin_elem.text else None
        
        # cp:endLifespanVersion
        end_elem = parcel_elem.find("CP:endLifespanVersion", namespaces=ns)
        end_val = end_elem.text.strip() if (end_elem is not None and end_elem.text) else None
        
        # cp:geometry – uložíme text z gml:posList uvnitř polygonu
        geom_elem = parcel_elem.find("CP:geometry/gml:Polygon/gml:exterior/gml:LinearRing/gml:posList", namespaces=ns)
        geometry_text = geom_elem.text.strip() if geom_elem is not None and geom_elem.text else None
        
        # cp:inspireId – získáme base:localId a base:namespace
        inspire_local = None
        inspire_namespace = None
        inspire_elem = parcel_elem.find("CP:inspireId/base:Identifier", namespaces=ns)
        if inspire_elem is not None:
            local_elem = inspire_elem.find("base:localId", namespaces=ns)
            ns_elem = inspire_elem.find("base:namespace", namespaces=ns)
            inspire_local = local_elem.text.strip() if local_elem is not None and local_elem.text else None
            inspire_namespace = ns_elem.text.strip() if ns_elem is not None and ns_elem.text else None
        
        # cp:label
        label_elem = parcel_elem.find("CP:label", namespaces=ns)
        label_val = label_elem.text.strip() if label_elem is not None and label_elem.text else None
        
        # cp:nationalCadastralReference
        ref_elem = parcel_elem.find("CP:nationalCadastralReference", namespaces=ns)
        ref_val = ref_elem.text.strip() if ref_elem is not None and ref_elem.text else None
        
        # cp:referencePoint – extrakce souřadnic z gml:Point/gml:pos
        ref_point_elem = parcel_elem.find("CP:referencePoint/gml:Point/gml:pos", namespaces=ns)
        if ref_point_elem is not None and ref_point_elem.text:
            coords = ref_point_elem.text.strip().split()
            if len(coords) >= 2:
                try:
                    x = float(coords[0])
                    y = float(coords[1])
                except ValueError:
                    x, y = None, None
            else:
                x, y = None, None
        else:
            x, y = None, None
        
        # cp:validFrom
        valid_from_elem = parcel_elem.find("CP:validFrom", namespaces=ns)
        valid_from_val = valid_from_elem.text.strip() if valid_from_elem is not None and valid_from_elem.text else None
        
        # cp:administrativeUnit – získáme hodnoty atributů xlink:href a xlink:title
        admin_elem = parcel_elem.find("CP:administrativeUnit", namespaces=ns)
        admin_href = admin_elem.get("{http://www.w3.org/1999/xlink}href") if admin_elem is not None else None
        admin_title = admin_elem.get("{http://www.w3.org/1999/xlink}title") if admin_elem is not None else None
        
        # cp:zoning – získáme hodnoty atributů xlink:href a xlink:title
        zoning_elem = parcel_elem.find("CP:zoning", namespaces=ns)
        zoning_href = zoning_elem.get("{http://www.w3.org/1999/xlink}href") if zoning_elem is not None else None
        zoning_title = zoning_elem.get("{http://www.w3.org/1999/xlink}title") if zoning_elem is not None else None
        
        # Transformace souřadnic z EPSG:5514 do EPSG:4326
        if x is not None and y is not None:
            transformer = Transformer.from_crs("EPSG:5514", "EPSG:4326", always_xy=True)
            lon, lat = transformer.transform(x, y)
        else:
            lon, lat = None, None
        
        record = {
            "kat_uzemi": zoning_name,
            "upper_zoning_id": upper_zoning_id,
            "parcel_number": parcel_number,
            "gml_id": gml_id,
            "areaValue_m2": area_val,
            "beginLifespanVersion": begin_val,
            "endLifespanVersion": end_val,
            "geometry": geometry_text,
            "inspire_localId": inspire_local,
            "inspire_namespace": inspire_namespace,
            "label": label_val,
            "nationalCadastralReference": ref_val,
            "refPoint_x": x,
            "refPoint_y": y,
            "refPoint_lon": lon,
            "refPoint_lat": lat,
            "validFrom": valid_from_val,
            "administrativeUnit_href": admin_href,
            "administrativeUnit_title": admin_title,
            "zoning_href": zoning_href,
            "zoning_title": zoning_title
        }
        records.append(record)
    
    if records:
        df = pd.DataFrame(records)
        print("\nDataFrame s informacemi o parcele:")
        print(df)
    else:
        print("Nebyly nalezeny žádné parcelní záznamy.")
    
    return df

# ==============================================================================
# Hlavní spuštění
# ==============================================================================
if __name__ == "__main__":
    # Zadání vstupních údajů
    kat_uzemi = "Malešice"   # Název katastrálního území
    parc_cislo = "875"      # Parcelní číslo
    
    # Získání dat o parcele a vytvoření DataFrame
    df_parcel = get_parcel_data(kat_uzemi, parc_cislo)
    print(df_parcel)


    # Uložení DataFrame do CSV pro pozdější import do databáze
    #csv_filename = "parcel_data.csv"
    #df_parcel.to_csv(csv_filename, index=False)
    #print(f"\nData byla uložena do souboru '{os.path.abspath(csv_filename)}'.")


In [65]:
# Testovani API CUZK pro parcely

import requests
import json

API_KEY = "2aB6dTFdcNhLosNIiHEwyhBXt8k26V"     # váš klíč, čistě ASCII
HEADERS = {"ApiKey": API_KEY, "Accept": "application/json"}

params = {
    "kmenoveCisloParcely":   2460,    # část před lomítkem
    "druhCislovaniParcely":   2,      # 1 = stavební, 2 = parcelní
    "poddeleniCislaParcely": 70,      # část za lomítkem
    "typParcely":           "PKN",    # PKN = pozemek, PZE = právo zástavní
    "kodKatastralnihoUzemi":762733    # kód KÚ
}

r = requests.get(
    "https://api-kn.cuzk.gov.cz/api/v1/Parcely/Vyhledani",
    headers=HEADERS,
    params=params
)
r.raise_for_status()
parcel = r.json()["data"][0]

# Strukturovaný výpis
print("\n=== Základní údaje o parcele ===")
print(f"ID parcely:              {parcel['id']}")
print(f"Typ parcely:             {parcel['typParcely']}")
print(f"Kód KÚ:                  {parcel['katastralniUzemi']['kod']} — {parcel['katastralniUzemi']['nazev']}")
print(f"Parcelní číslo:          {parcel['kmenoveCisloParcely']}/{parcel['poddeleniCislaParcely']}")
print(f"Výměra (m²):             {parcel.get('vymera', '—')}")

print("\n=== LV (List vlastnictví) ===")
lv = parcel.get("lv", {})
print(f"  LV ID:                 {lv.get('id', '—')}")
print(f"  Číslo listu vlast.:    {lv.get('cislo', '—')}")
print(f"  KÚ LV:                 {lv.get('katastralniUzemi', {}).get('kod', '—')}")

print("\n=== BPEJ (půdní druhy) ===")
for b in parcel.get("bpej", []):
    print(f"  • Kód: {b.get('kod')}  Výmera: {b.get('vymera')} m²")

print("\n=== Řízení / plomby ===")
for rizo in parcel.get("rizeniPlomby", []):
    print(f"  • ID řízení: {rizo.get('id')}, typ: {rizo.get('typRizeni')}, rok: {rizo.get('rok')}, Číslo: {rizo.get('poradoveCislo')}")

# Pokud byste chtěli vidět celý záznam v pěkném JSON formátu:
print("\n=== Suplný JSON záznam ===")
print(json.dumps(parcel, ensure_ascii=False, indent=2))





=== Základní údaje o parcele ===
ID parcely:              93678394010
Typ parcely:             PKN
Kód KÚ:                  762733 — Škvorec
Parcelní číslo:          2460/70
Výměra (m²):             25890

=== LV (List vlastnictví) ===
  LV ID:                 63678714010
  Číslo listu vlast.:    1540
  KÚ LV:                 762733

=== BPEJ (půdní druhy) ===
  • Kód: 21100  Výmera: 6129 m²
  • Kód: 22604  Výmera: 11 m²
  • Kód: 22611  Výmera: 16320 m²
  • Kód: 22641  Výmera: 3430 m²

=== Řízení / plomby ===

=== Suplný JSON záznam ===
{
  "id": 93678394010,
  "typParcely": "PKN",
  "druhCislovaniParcely": 2,
  "kmenoveCisloParcely": 2460,
  "poddeleniCislaParcely": 70,
  "katastralniUzemi": {
    "kod": 762733,
    "nazev": "Škvorec"
  },
  "zdrojParcelyZE": null,
  "katastralniUzemiPuvodni": null,
  "dilParcely": null,
  "vymera": 25890,
  "lv": {
    "id": 63678714010,
    "cislo": 1540,
    "katastralniUzemi": {
      "kod": 762733,
      "nazev": "Škvorec"
    }
  },
  "mapovyLi

In [ ]:
# Testování tahání info z KN podle ID pacely
import requests

API_KEY = "2aB6dTFdcNhLosNIiHEwyhBXt8k26V"   
HEADERS = {"ApiKey": API_KEY, "Accept": "application/json"}

# 1) Přímo podle ID
parc_id = 93678394010
r = requests.get(
    f"https://api-kn.cuzk.gov.cz/api/v1/Parcely/{parc_id}",
    headers=HEADERS
)
r.raise_for_status()
detail = r.json().get("data")    # nebo r.json()["data"]
print(detail)


{'id': 93678394010, 'typParcely': 'PKN', 'druhCislovaniParcely': 2, 'kmenoveCisloParcely': 2460, 'poddeleniCislaParcely': 70, 'katastralniUzemi': {'kod': 762733, 'nazev': 'Škvorec'}, 'zdrojParcelyZE': None, 'katastralniUzemiPuvodni': None, 'dilParcely': None, 'vymera': 25890, 'lv': {'id': 63678714010, 'cislo': 1540, 'katastralniUzemi': {'kod': 762733, 'nazev': 'Škvorec'}}, 'mapovyList': {'kod': 6780, 'oznaceni': 'DKM'}, 'zpusobUrceniVymery': {'kod': 2, 'nazev': 'Ze souřadnic v S-JTSK'}, 'druhPozemku': {'kod': 2, 'nazev': 'orná půda'}, 'stavba': None, 'pravoStavby': None, 'definicniBod': {'id': 59903633010, 'x': -720798.06, 'y': -1048996.67}, 'zpusobVyuziti': None, 'zpusobyOchrany': [{'kod': 27, 'nazev': 'zemědělský půdní fond'}], 'bpej': [{'kod': 21100, 'vymera': 6129}, {'kod': 22604, 'vymera': 11}, {'kod': 22611, 'vymera': 16320}, {'kod': 22641, 'vymera': 3430}], 'rizeniPlomby': []}


In [8]:
# *** 1 *** DOPLŇOVÁNÍ INFORMACÍ Z KATASTRU NEMOVITOSTÍ POMOCÍ SLUŽBY WFS K POZEMKŮM DO DB - FUNKČNÍ***
#/////////////////////////////////////////////////////////////////////////////////////////
# VSTUPNÍ DATA = Valuo_data
# výstupní data v KN_parcel_data
#
#/////////////////////////////////////////////////////////////////////////////////////////

import requests
from lxml import etree
from pyproj import Transformer
import pandas as pd
import os
import urllib.parse
import logging
from sqlalchemy import create_engine, text

# ==============================================================================
# Nastavení logování: výpis na konzoli i do souboru "process_log.txt"
# ==============================================================================
logger = logging.getLogger("ParcelProcessing")
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

# Handler pro konzoli
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
console_handler.setFormatter(formatter)
logger.addHandler(console_handler)

# Handler pro soubor
file_handler = logging.FileHandler("process_log.txt", encoding="utf-8")
file_handler.setLevel(logging.DEBUG)
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

# ==============================================================================
# Funkce pro parsování adresy podle adresa a popis
# ==============================================================================
def parse_address(adresa, popis):
    """
    Vrátí tuple (kat_uzemi, parc_cislo) na základě informací ve sloupci 'adresa'.
    
    Použije se primárně hodnota ze sloupce adresa (očekává se formát "<kat_uzemi> <parcelní číslo>").
    Pokud zároveň sloupec 'popis' obsahuje slovo "stavební" (case-insensitive),
    potom se parcelní číslo upraví tak, že pokud nezačíná "st. " (s mezerou za tečkou), přidá se předpona "st. ".
    
    Příklad:
      adresa = "Dolní Břežany 273"
      popis  = "stavební č. 273 Dolní Břežany (součástí je stavba rozestav. budova)"
      Vrátí: ("Dolní Břežany", "st. 273")
    """
    try:
        kat_uzemi, parc_cislo = adresa.rsplit(" ", 1)
    except ValueError:
        raise ValueError(f"Adresa '{adresa}' nemá očekávaný formát.")
    
    if popis and "stavební" in popis.lower():
        # Pokud parcelní číslo již nezačíná s "st. ", přidáme předponu "st. " s mezerou
        if not parc_cislo.lower().startswith("st. "):
            parc_cislo = "st. " + parc_cislo
    return kat_uzemi, parc_cislo

# ==============================================================================
# Funkce pro získání kódu katastrálního území (UPPER_ZONING_ID) podle názvu
# ==============================================================================
def get_zoning_code(zoning_name):
    base_url = "https://services.cuzk.cz/wfs/inspire-CP-wfs.asp?"
    params = {
        "service": "WFS",
        "version": "2.0.0",
        "request": "GetFeature",
        "storedQuery_id": "GetZoningByName",
        "ZONING_NAME": zoning_name
    }
    logger.info(f"\nDotazuji katastrální území (GetZoningByName) pro: {zoning_name}")
    response = requests.get(base_url, params=params)
    logger.debug(f"HTTP status (GetZoningByName): {response.status_code}")
    
    filename = "zoning_response.xml"
    with open(filename, "wb") as f:
        f.write(response.content)
    logger.debug(f"XML odpověď (GetZoningByName) uložena: {os.path.abspath(filename)}")
    
    try:
        tree = etree.fromstring(response.content)
    except Exception as e:
        raise Exception(f"Chyba při parsování XML (GetZoningByName): {e}")
    
    ns = {
        "CP": "http://inspire.ec.europa.eu/schemas/cp/4.0",
        "base": "http://inspire.ec.europa.eu/schemas/base/3.3",
        "wfs": "http://www.opengis.net/wfs/2.0"
    }
    zoning_id_elem = tree.find(".//CP:CadastralZoning/CP:inspireId/base:Identifier/base:localId", namespaces=ns)
    if zoning_id_elem is not None and zoning_id_elem.text:
        zoning_id_full = zoning_id_elem.text.strip()
        logger.info(f"Nalezený identifikátor (full): {zoning_id_full}")
        zoning_id = zoning_id_full[3:] if zoning_id_full.startswith("CZ.") else zoning_id_full
        logger.info(f"Vyhodnocený kód katastrálního území (UPPER_ZONING_ID): {zoning_id}")
        return zoning_id
    else:
        logger.info("Nebyl nalezen identifikátor katastrálního území.")
        return None

# ==============================================================================
# Funkce pro získání dat o parcele a vytvoření DataFrame pro jeden záznam
# ==============================================================================
def get_parcel_data(zoning_name, parcel_number):
    """
    Na základě názvu katastrálního území a parcelního čísla získá podrobné informace o parcele
    prostřednictvím uloženého dotazu GetParcel a vrátí DataFrame s extrahovanými daty.
    Pokud nejsou nalezeny žádné záznamy, diagnostikuje možné důvody a vrátí prázdný DataFrame.
    """
    upper_zoning_id = get_zoning_code(zoning_name)
    if not upper_zoning_id:
        raise Exception("Nepodařilo se získat kód katastrálního území.")
    
    base_url = "https://services.cuzk.cz/wfs/inspire-CP-wfs.asp?"
    params = {
        "service": "WFS",
        "version": "2.0.0",
        "request": "GetFeature",
        "storedQuery_id": "GetParcel",
        "UPPER_ZONING_ID": upper_zoning_id,
        "TEXT": parcel_number
    }
    
    logger.info("\nSestavuji dotaz pomocí storedQuery 'GetParcel'...")
    logger.info(f"Základní URL: {base_url}")
    for k, v in params.items():
        logger.info(f"  {k}: {v}")
    
    response = requests.get(base_url, params=params)
    logger.info(f"HTTP status kód odpovědi (GetParcel): {response.status_code}")
    
    filename = "parcel_response.xml"
    with open(filename, "wb") as f:
        f.write(response.content)
    logger.info(f"XML odpověď (GetParcel) uložena: {os.path.abspath(filename)}")
    
    try:
        tree = etree.fromstring(response.content)
    except Exception as e:
        raise Exception(f"Chyba při parsování XML (GetParcel): {e}")
    
    ns = {
        "wfs": "http://www.opengis.net/wfs/2.0",
        "gml": "http://www.opengis.net/gml/3.2",
        "CP": "http://inspire.ec.europa.eu/schemas/cp/4.0",
        "base": "http://inspire.ec.europa.eu/schemas/base/3.3"
    }
    
    members = tree.xpath("//wfs:member", namespaces=ns)
    logger.info(f"\nNalezeno {len(members)} feature (member) pro parcelu {parcel_number}.")
    
    records = []
    for mem in members:
        parcel_elem = mem.find(".//CP:CadastralParcel", namespaces=ns)
        if parcel_elem is None:
            continue
        
        gml_id = parcel_elem.get("{http://www.opengis.net/gml/3.2}id")
        
        area_elem = parcel_elem.find("CP:areaValue", namespaces=ns)
        try:
            area_val = float(area_elem.text.strip()) if area_elem is not None and area_elem.text else None
        except ValueError:
            area_val = None
        
        begin_elem = parcel_elem.find("CP:beginLifespanVersion", namespaces=ns)
        begin_val = begin_elem.text.strip() if begin_elem is not None and begin_elem.text else None
        
        end_elem = parcel_elem.find("CP:endLifespanVersion", namespaces=ns)
        end_val = end_elem.text.strip() if (end_elem is not None and end_elem.text) else None
        
        geom_elem = parcel_elem.find("CP:geometry/gml:Polygon/gml:exterior/gml:LinearRing/gml:posList", namespaces=ns)
        geometry_text = geom_elem.text.strip() if geom_elem is not None and geom_elem.text else None
        
        inspire_local = None
        inspire_namespace = None
        inspire_elem = parcel_elem.find("CP:inspireId/base:Identifier", namespaces=ns)
        if inspire_elem is not None:
            local_elem = inspire_elem.find("base:localId", namespaces=ns)
            ns_elem = inspire_elem.find("base:namespace", namespaces=ns)
            inspire_local = local_elem.text.strip() if local_elem is not None and local_elem.text else None
            inspire_namespace = ns_elem.text.strip() if ns_elem is not None and ns_elem.text else None
        
        label_elem = parcel_elem.find("CP:label", namespaces=ns)
        label_val = label_elem.text.strip() if label_elem is not None and label_elem.text else None
        
        ref_elem = parcel_elem.find("CP:nationalCadastralReference", namespaces=ns)
        ref_val = ref_elem.text.strip() if ref_elem is not None and ref_elem.text else None
        
        ref_point_elem = parcel_elem.find("CP:referencePoint/gml:Point/gml:pos", namespaces=ns)
        if ref_point_elem is not None and ref_point_elem.text:
            coords = ref_point_elem.text.strip().split()
            if len(coords) >= 2:
                try:
                    x = float(coords[0])
                    y = float(coords[1])
                except ValueError:
                    x, y = None, None
            else:
                x, y = None, None
        else:
            x, y = None, None
        
        valid_from_elem = parcel_elem.find("CP:validFrom", namespaces=ns)
        valid_from_val = valid_from_elem.text.strip() if valid_from_elem is not None and valid_from_elem.text else None
        
        admin_elem = parcel_elem.find("CP:administrativeUnit", namespaces=ns)
        admin_href = admin_elem.get("{http://www.w3.org/1999/xlink}href") if admin_elem is not None else None
        admin_title = admin_elem.get("{http://www.w3.org/1999/xlink}title") if admin_elem is not None else None
        
        zoning_elem = parcel_elem.find("CP:zoning", namespaces=ns)
        zoning_href = zoning_elem.get("{http://www.w3.org/1999/xlink}href") if zoning_elem is not None else None
        zoning_title = zoning_elem.get("{http://www.w3.org/1999/xlink}title") if zoning_elem is not None else None
        
        if x is not None and y is not None:
            transformer = Transformer.from_crs("EPSG:5514", "EPSG:4326", always_xy=True)
            lon, lat = transformer.transform(x, y)
        else:
            lon, lat = None, None
        
        record = {
            "kat_uzemi": zoning_name,
            "upper_zoning_id": upper_zoning_id,
            "parcel_number": parcel_number,
            "gml_id": gml_id,
            "areaValue_m2": area_val,
            "beginLifespanVersion": begin_val,
            "endLifespanVersion": end_val,
            "geometry": geometry_text,
            "inspire_localId": inspire_local,
            "inspire_namespace": inspire_namespace,
            "label": label_val,
            "nationalCadastralReference": ref_val,
            "refPoint_x": x,
            "refPoint_y": y,
            "refPoint_lon": lon,
            "refPoint_lat": lat,
            "validFrom": valid_from_val,
            "administrativeUnit_href": admin_href,
            "administrativeUnit_title": admin_title,
            "zoning_href": zoning_href,
            "zoning_title": zoning_title
        }
        records.append(record)
    
    if records:
        df = pd.DataFrame(records)
        logger.info("\nDataFrame s informacemi o parcele:")
        logger.info(df)
    else:
        fc = tree.xpath("/wfs:FeatureCollection", namespaces={"wfs": "http://www.opengis.net/wfs/2.0"})
        if fc:
            fc_elem = fc[0]
            num_matched = fc_elem.get("numberMatched", "není uvedeno")
            num_returned = fc_elem.get("numberReturned", "není uvedeno")
            logger.info("\nFeatureCollection:")
            logger.info(f"  numberMatched = {num_matched}")
            logger.info(f"  numberReturned = {num_returned}")
        xml_preview = response.text[:500]
        logger.info("\nNáhled raw XML odpovědi:")
        logger.info(xml_preview)
        logger.info("\nMožné důvody nenalezení parcely:")
        logger.info("  1. Zadané katastrální území nebo parcelní číslo neodpovídá formátu dat v systému.")
        logger.info("  2. Daná parcela se v databázi nenachází (mimo digitální mapu).")
        logger.info("  3. Chyba ve filtru nebo nesprávné rozdělení vstupního řetězce.")
        df = pd.DataFrame()  # prázdný DataFrame
    return df

# ==============================================================================
# Hlavní spuštění: Načtení vstupních dat z DB, zpracování parcel, vložení do DB a souhrnné statistiky
# ==============================================================================
if __name__ == "__main__":
    from sqlalchemy import create_engine
    import urllib.parse
    
    # Připojení k DB Valuo pomocí SQLAlchemy
    params_conn = urllib.parse.quote_plus(
        "Driver={ODBC Driver 17 for SQL Server};"
        "Server=localhost;"
        "Database=VALUO;"
        "Trusted_Connection=yes;"
    )
    connection_url = f"mssql+pyodbc:///?odbc_connect={params_conn}"
    engine = create_engine(connection_url)
    
    # Načtení vstupních dat z tabulky Valuo_data, kde nemovitost = 'parcela'
    query = """
    SELECT id AS id_valuo, adresa, popis
    FROM Valuo_data 
    WHERE nemovitost = 'parcela'
      AND adresa <> 'Neznámá adresa'
      AND KN_WFS_info IS NULL
    """
    df_input = pd.read_sql(query, engine)
    logger.info("\nVstupní data z DB:")
    logger.info(df_input.head())
    
    total_db_records = len(df_input)
    queries_attempted = 0
    successful_queries = 0
    unsuccessful_ids = []  # seznam id_valuo, pro které dotaz nenašel parcelu
    
    df_list = []
    
    for idx, row in df_input.iterrows():
        id_valuo = row["id_valuo"]
        adresa = row["adresa"].strip()
        popis = row["popis"].strip() if row["popis"] is not None else ""
        if not adresa:
            continue
        try:
            # Rozhodneme, zda použít formát podle popisu (pro stavební parcely) nebo podle adresy
            if popis and "stavební" in popis.lower():
                kat_uzemi, parc_cislo = parse_address(adresa, popis)
            else:
                kat_uzemi, parc_cislo = parse_address(adresa, "")
            queries_attempted += 1
        except Exception as e:
            logger.error(f"Adresa '{adresa}' nemá očekávaný formát: {e}. Přeskakuji.")
            continue
        
        logger.info(f"\nZpracovávám záznam id_valuo={id_valuo}, adresa='{adresa}'")
        try:
            df_parcel = get_parcel_data(kat_uzemi, parc_cislo)
            if df_parcel is not None and not df_parcel.empty:
                successful_queries += 1
                df_parcel["id_valuo"] = id_valuo  # propojení s Valuo_data
                df_list.append(df_parcel)
            else:
                logger.info(f"Nebyly nalezeny data o parcele pro id_valuo={id_valuo}.")
                unsuccessful_ids.append(id_valuo)
        except Exception as e:
            logger.error(f"Chyba při zpracování id_valuo={id_valuo}: {e}")
            unsuccessful_ids.append(id_valuo)
    
    if df_list:
        df_all = pd.concat(df_list, ignore_index=True)
        logger.info("\nKonečný DataFrame se všemi získanými daty:")
        logger.info(df_all)
        
        # Vložení dat do tabulky KN_parcel_data
        df_all.to_sql("KN_parcel_data", con=engine, if_exists="append", index=False)
        inserted_rows = len(df_all)
        logger.info(f"\nData byla vložena do tabulky KN_parcel_data, počet vložených řádků: {inserted_rows}.")
    else:
        df_all = pd.DataFrame()
        logger.info("Nebyla získána žádná data o parcelách.")
    
    # Aktualizace tabulky Valuo_data:
    # Pro záznamy, u kterých byly získány data, nastavíme KN_WFS_info = 1,
    # a pro ty, kde nebyly data nalezeny, nastavíme KN_WFS_info = 0.
    successful_ids = df_all["id_valuo"].unique().tolist() if not df_all.empty else []
    # Předpokládáme, že všechny záznamy, které byly načteny z DB, jsou v df_input.
    all_ids = df_input["id_valuo"].unique().tolist()
    
    
    # Aktualizace úspěšných záznamů: KN_WFS_info = 1
    updated_success = 0
    if successful_ids:
        id_str_success = ",".join(str(i) for i in successful_ids)
        update_query_success = f"UPDATE Valuo_data SET KN_WFS_info = 1 WHERE id IN ({id_str_success})"
        with engine.begin() as conn:
            result_success = conn.execute(text(update_query_success))
            updated_success = result_success.rowcount if result_success.rowcount is not None else len(successful_ids)
        logger.info(f"\nTabulka Valuo_data aktualizována: pro záznamy s id IN ({id_str_success}) bylo nastaveno KN_WFS_info = 1.")
    
    # Aktualizace neúspěšných záznamů: KN_WFS_info = 0
    updated_unsuccess = 0
    if unsuccessful_ids:
        id_str_unsuccess = ",".join(str(i) for i in unsuccessful_ids)
        update_query_unsuccess = f"UPDATE Valuo_data SET KN_WFS_info = 0 WHERE id IN ({id_str_unsuccess})"
        with engine.begin() as conn:
            result_unsuccess = conn.execute(text(update_query_unsuccess))
            updated_unsuccess = result_unsuccess.rowcount if result_unsuccess.rowcount is not None else len(unsuccessful_ids)
        logger.info(f"\nTabulka Valuo_data aktualizována: pro záznamy s id IN ({id_str_unsuccess}) bylo nastaveno KN_WFS_info = 0.")
    
    # Souhrnné statistiky
    summary = "\n" + "="*30 + " SOUHRN STATISTIK " + "="*30 + "\n"
    summary += f"Celkový počet načtených záznamů z DB Valuo_data: {len(all_ids)}\n"
    summary += f"Počet dotazů odeslaných do katastru: {queries_attempted}\n"
    summary += f"Počet úspěšně zodpovězených dotazů (parcely získány): {successful_queries}\n"
    summary += f"Celkový počet nových řádků vložených do KN_parcel_data: {inserted_rows}\n"
    summary += f"Počet aktualizovaných řádků v tabulce Valuo_data (KN_WFS_info=1): {updated_success}\n"
    summary += f"Počet aktualizovaných řádků v tabulce Valuo_data (KN_WFS_info=0): {updated_unsuccess}\n"
    summary += "="*80 + "\n"
    logger.info(summary)
    print(summary)



2025-05-29 08:36:40,022 - INFO - 
Vstupní data z DB:
2025-05-29 08:36:40,022 - INFO - 
Vstupní data z DB:
2025-05-29 08:36:40,022 - INFO - 
Vstupní data z DB:
2025-05-29 08:36:40,024 - INFO - Empty DataFrame
Columns: [id_valuo, adresa, popis]
Index: []
2025-05-29 08:36:40,024 - INFO - Empty DataFrame
Columns: [id_valuo, adresa, popis]
Index: []
2025-05-29 08:36:40,024 - INFO - Empty DataFrame
Columns: [id_valuo, adresa, popis]
Index: []
2025-05-29 08:36:40,027 - INFO - Nebyla získána žádná data o parcelách.
2025-05-29 08:36:40,027 - INFO - Nebyla získána žádná data o parcelách.
2025-05-29 08:36:40,027 - INFO - Nebyla získána žádná data o parcelách.


NameError: name 'inserted_rows' is not defined

In [7]:
# *** 2 *** DOPLŇOVÁNÍ INFORMACÍ Z KATASTRU NEMOVITOSTÍ POMOCÍ SLUŽBY WFS K POZEMKŮM PRO BYTY (TAHANI POZEMKU PRO BYTY)   FUNKČNÍ***
#/////////////////////////////////////////////////////////////////////////////////////////
# VSTUPNÍ DATA = Valuo_data
# výstupní data v KN_parcel_data
# Melo by to fungovat i pro pozemky
#
#/////////////////////////////////////////////////////////////////////////////////////////



import requests
from lxml import etree
from pyproj import Transformer
import pandas as pd
import os
import urllib.parse
import logging
import re  # Import regulárních výrazů
from sqlalchemy import create_engine, text


# ==============================================================================
# Nastavení logování: výpis na konzoli i do souboru "process_log.txt"
# ==============================================================================
logger = logging.getLogger("ParcelProcessing")
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

# Handler pro konzoli
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
console_handler.setFormatter(formatter)
logger.addHandler(console_handler)

# Handler pro soubor
file_handler = logging.FileHandler("process_log.txt", encoding="utf-8")
file_handler.setLevel(logging.DEBUG)
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

# ==============================================================================
# Funkce pro parsování adresy podle adresa a popis (původní logika)
# ==============================================================================
def parse_address(adresa, popis):
    """
    Vrátí tuple (kat_uzemi, parc_cislo) na základě informací ve sloupci 'adresa'.
    
    Použije se primárně hodnota ze sloupce adresa (očekává se formát "<kat_uzemi> <parcelní číslo>").
    Pokud zároveň sloupec 'popis' obsahuje slovo "stavební" (case-insensitive),
    potom se parcelní číslo upraví tak, že pokud nezačíná "st. " (s mezerou za tečkou), přidá se předpona "st. ".
    
    Příklad:
      adresa = "Dolní Břežany 273"
      popis  = "stavební č. 273 Dolní Břežany (součástí je stavba rozestav. budova)"
      Vrátí: ("Dolní Břežany", "st. 273")
    """
    try:
        kat_uzemi, parc_cislo = adresa.rsplit(" ", 1)
    except ValueError:
        raise ValueError(f"Adresa '{adresa}' nemá očekávaný formát.")
    
    if popis and "stavební" in popis.lower():
        # Pokud parcelní číslo již nezačíná s "st. ", přidáme předponu "st. " s mezerou
        if not parc_cislo.lower().startswith("st. "):
            parc_cislo = "st. " + parc_cislo
    return kat_uzemi, parc_cislo

# ==============================================================================
# Nová funkce pro extrakci parcelních čísel a katastrálních území z pole "popis"
# ==============================================================================
def extract_parcels_from_popis(popis, default_kat_uzemi=None):
    """
    Extrahuje seznam dvojic (kat_uzemi, parc_cislo) z textu v poli 'popis'.
    
    Vyhledá část textu za výrazem "na parcele" až do výskytu klíčových slov ("podíl" nebo "(součástí")
    nebo do konce řetězce, pokud klíčová slova chybí.
    
    Následně rozdělí text podle čárek a pro každý segment pomocí regulárního výrazu extrahuje:
      - volitelný prefix "st. ",
      - parcelní číslo (čísla, případně s lomítkem a dalšími čísly),
      - název katastrálního území (pokud není uvedeno, použije se předchozí hodnota nebo default_kat_uzemi).
      
    Pokud byl prefix "st. " nalezen, výsledné parcelní číslo bude obsahovat tento prefix.
    """
    import re
    # Rozšířený regex: zachytí text za "na parcele" až do výskytu "podíl", "(součástí" nebo konce řetězce.
    match = re.search(r"na parcele\s+(.+?)(?=\s*(podíl|\(součástí)|$)", popis, re.IGNORECASE)
    if not match:
        return []
    
    # Odebere případné koncové čárky a nepotřebné mezery
    parcel_section = match.group(1).strip().rstrip(",")
    segments = [seg.strip() for seg in parcel_section.split(",") if seg.strip()]
    
    results = []
    last_kat = default_kat_uzemi  # počáteční hodnota katastrálního území
    
    for seg in segments:
        # Regex: volitelný prefix "st. ", číslo (možno s lomítkem) a volitelný název katastrálního území
        seg_match = re.match(r"(?i)^(st\.\s*)?(\d+(?:/\d+)?)(?:\s+(.+))?$", seg)
        if seg_match:
            prefix = seg_match.group(1) or ""
            parc_number = seg_match.group(2)
            # Ujistíme se, že pokud je prefix přítomen, má formu "st. " (s mezerou)
            if prefix and not prefix.lower().startswith("st. "):
                prefix = "st. "
            parc_cislo = f"{prefix}{parc_number}".strip()
            
            # Pokud je uveden název katastrálního území, použijeme jej a aktualizujeme last_kat.
            # Jinak použijeme poslední známou hodnotu (nebo default).
            kat = seg_match.group(3).strip() if seg_match.group(3) else None
            if kat:
                last_kat = kat
            else:
                kat = last_kat
            
            results.append((kat, parc_cislo))
    
    return results



# ==============================================================================
# Funkce pro získání kódu katastrálního území (UPPER_ZONING_ID) podle názvu
# ==============================================================================
def get_zoning_code(zoning_name):
    base_url = "https://services.cuzk.cz/wfs/inspire-CP-wfs.asp?"
    params = {
        "service": "WFS",
        "version": "2.0.0",
        "request": "GetFeature",
        "storedQuery_id": "GetZoningByName",
        "ZONING_NAME": zoning_name
    }
    logger.info(f"\nDotazuji katastrální území (GetZoningByName) pro: {zoning_name}")
    response = requests.get(base_url, params=params)
    logger.debug(f"HTTP status (GetZoningByName): {response.status_code}")
    
    filename = "zoning_response.xml"
    with open(filename, "wb") as f:
        f.write(response.content)
    logger.debug(f"XML odpověď (GetZoningByName) uložena: {os.path.abspath(filename)}")
    
    try:
        tree = etree.fromstring(response.content)
    except Exception as e:
        raise Exception(f"Chyba při parsování XML (GetZoningByName): {e}")
    
    ns = {
        "CP": "http://inspire.ec.europa.eu/schemas/cp/4.0",
        "base": "http://inspire.ec.europa.eu/schemas/base/3.3",
        "wfs": "http://www.opengis.net/wfs/2.0"
    }
    zoning_id_elem = tree.find(".//CP:CadastralZoning/CP:inspireId/base:Identifier/base:localId", namespaces=ns)
    if zoning_id_elem is not None and zoning_id_elem.text:
        zoning_id_full = zoning_id_elem.text.strip()
        logger.info(f"Nalezený identifikátor (full): {zoning_id_full}")
        zoning_id = zoning_id_full[3:] if zoning_id_full.startswith("CZ.") else zoning_id_full
        logger.info(f"Vyhodnocený kód katastrálního území (UPPER_ZONING_ID): {zoning_id}")
        return zoning_id
    else:
        logger.info("Nebyl nalezen identifikátor katastrálního území.")
        return None

# ==============================================================================
# Funkce pro získání dat o parcele a vytvoření DataFrame pro jeden záznam
# ==============================================================================
def get_parcel_data(zoning_name, parcel_number):
    """
    Na základě názvu katastrálního území a parcelního čísla získá podrobné informace o parcele
    prostřednictvím uloženého dotazu GetParcel a vrátí DataFrame s extrahovanými daty.
    Pokud nejsou nalezeny žádné záznamy, diagnostikuje možné důvody a vrátí prázdný DataFrame.
    """
    upper_zoning_id = get_zoning_code(zoning_name)
    if not upper_zoning_id:
        raise Exception("Nepodařilo se získat kód katastrálního území.")
    
    base_url = "https://services.cuzk.cz/wfs/inspire-CP-wfs.asp?"
    params = {
        "service": "WFS",
        "version": "2.0.0",
        "request": "GetFeature",
        "storedQuery_id": "GetParcel",
        "UPPER_ZONING_ID": upper_zoning_id,
        "TEXT": parcel_number
    }
    
    logger.info("\nSestavuji dotaz pomocí storedQuery 'GetParcel'...")
    logger.info(f"Základní URL: {base_url}")
    for k, v in params.items():
        logger.info(f"  {k}: {v}")
    
    response = requests.get(base_url, params=params)
    logger.info(f"HTTP status kód odpovědi (GetParcel): {response.status_code}")
    
    filename = "parcel_response.xml"
    with open(filename, "wb") as f:
        f.write(response.content)
    logger.info(f"XML odpověď (GetParcel) uložena: {os.path.abspath(filename)}")
    
    try:
        tree = etree.fromstring(response.content)
    except Exception as e:
        raise Exception(f"Chyba při parsování XML (GetParcel): {e}")
    
    ns = {
        "wfs": "http://www.opengis.net/wfs/2.0",
        "gml": "http://www.opengis.net/gml/3.2",
        "CP": "http://inspire.ec.europa.eu/schemas/cp/4.0",
        "base": "http://inspire.ec.europa.eu/schemas/base/3.3"
    }
    
    members = tree.xpath("//wfs:member", namespaces=ns)
    logger.info(f"\nNalezeno {len(members)} feature (member) pro parcelu {parcel_number}.")
    
    records = []
    for mem in members:
        parcel_elem = mem.find(".//CP:CadastralParcel", namespaces=ns)
        if parcel_elem is None:
            continue
        
        gml_id = parcel_elem.get("{http://www.opengis.net/gml/3.2}id")
        
        area_elem = parcel_elem.find("CP:areaValue", namespaces=ns)
        try:
            area_val = float(area_elem.text.strip()) if area_elem is not None and area_elem.text else None
        except ValueError:
            area_val = None
        
        begin_elem = parcel_elem.find("CP:beginLifespanVersion", namespaces=ns)
        begin_val = begin_elem.text.strip() if begin_elem is not None and begin_elem.text else None
        
        end_elem = parcel_elem.find("CP:endLifespanVersion", namespaces=ns)
        end_val = end_elem.text.strip() if (end_elem is not None and end_elem.text) else None
        
        geom_elem = parcel_elem.find("CP:geometry/gml:Polygon/gml:exterior/gml:LinearRing/gml:posList", namespaces=ns)
        geometry_text = geom_elem.text.strip() if geom_elem is not None and geom_elem.text else None
        
        inspire_local = None
        inspire_namespace = None
        inspire_elem = parcel_elem.find("CP:inspireId/base:Identifier", namespaces=ns)
        if inspire_elem is not None:
            local_elem = inspire_elem.find("base:localId", namespaces=ns)
            ns_elem = inspire_elem.find("base:namespace", namespaces=ns)
            inspire_local = local_elem.text.strip() if local_elem is not None and local_elem.text else None
            inspire_namespace = ns_elem.text.strip() if ns_elem is not None and ns_elem.text else None
        
        label_elem = parcel_elem.find("CP:label", namespaces=ns)
        label_val = label_elem.text.strip() if label_elem is not None and label_elem.text else None
        
        ref_elem = parcel_elem.find("CP:nationalCadastralReference", namespaces=ns)
        ref_val = ref_elem.text.strip() if ref_elem is not None and ref_elem.text else None
        
        ref_point_elem = parcel_elem.find("CP:referencePoint/gml:Point/gml:pos", namespaces=ns)
        if ref_point_elem is not None and ref_point_elem.text:
            coords = ref_point_elem.text.strip().split()
            if len(coords) >= 2:
                try:
                    x = float(coords[0])
                    y = float(coords[1])
                except ValueError:
                    x, y = None, None
            else:
                x, y = None, None
        else:
            x, y = None, None
        
        valid_from_elem = parcel_elem.find("CP:validFrom", namespaces=ns)
        valid_from_val = valid_from_elem.text.strip() if valid_from_elem is not None and valid_from_elem.text else None
        
        admin_elem = parcel_elem.find("CP:administrativeUnit", namespaces=ns)
        admin_href = admin_elem.get("{http://www.w3.org/1999/xlink}href") if admin_elem is not None else None
        admin_title = admin_elem.get("{http://www.w3.org/1999/xlink}title") if admin_elem is not None else None
        
        zoning_elem = parcel_elem.find("CP:zoning", namespaces=ns)
        zoning_href = zoning_elem.get("{http://www.w3.org/1999/xlink}href") if zoning_elem is not None else None
        zoning_title = zoning_elem.get("{http://www.w3.org/1999/xlink}title") if zoning_elem is not None else None
        
        if x is not None and y is not None:
            transformer = Transformer.from_crs("EPSG:5514", "EPSG:4326", always_xy=True)
            lon, lat = transformer.transform(x, y)
        else:
            lon, lat = None, None
        
        record = {
            "kat_uzemi": zoning_name,
            "upper_zoning_id": upper_zoning_id,
            "parcel_number": parcel_number,
            "gml_id": gml_id,
            "areaValue_m2": area_val,
            "beginLifespanVersion": begin_val,
            "endLifespanVersion": end_val,
            "geometry": geometry_text,
            "inspire_localId": inspire_local,
            "inspire_namespace": inspire_namespace,
            "label": label_val,
            "nationalCadastralReference": ref_val,
            "refPoint_x": x,
            "refPoint_y": y,
            "refPoint_lon": lon,
            "refPoint_lat": lat,
            "validFrom": valid_from_val,
            "administrativeUnit_href": admin_href,
            "administrativeUnit_title": admin_title,
            "zoning_href": zoning_href,
            "zoning_title": zoning_title
        }
        records.append(record)
    
    if records:
        df = pd.DataFrame(records)
        logger.info("\nDataFrame s informacemi o parcele:")
        logger.info(df)
    else:
        fc = tree.xpath("/wfs:FeatureCollection", namespaces={"wfs": "http://www.opengis.net/wfs/2.0"})
        if fc:
            fc_elem = fc[0]
            num_matched = fc_elem.get("numberMatched", "není uvedeno")
            num_returned = fc_elem.get("numberReturned", "není uvedeno")
            logger.info("\nFeatureCollection:")
            logger.info(f"  numberMatched = {num_matched}")
            logger.info(f"  numberReturned = {num_returned}")
        xml_preview = response.text[:500]
        logger.info("\nNáhled raw XML odpovědi:")
        logger.info(xml_preview)
        logger.info("\nMožné důvody nenalezení parcely:")
        logger.info("  1. Zadané katastrální území nebo parcelní číslo neodpovídá formátu dat v systému.")
        logger.info("  2. Daná parcela se v databázi nenachází (mimo digitální mapu).")
        logger.info("  3. Chyba ve filtru nebo nesprávné rozdělení vstupního řetězce.")
        df = pd.DataFrame()  # prázdný DataFrame
    return df

# ==============================================================================
# Hlavní spuštění: Načtení vstupních dat z DB, zpracování parcel, vložení do DB a souhrnné statistiky
# ==============================================================================
if __name__ == "__main__":
    from sqlalchemy import create_engine
    import urllib.parse
    
    # Připojení k DB Valuo pomocí SQLAlchemy
    params_conn = urllib.parse.quote_plus(
        "Driver={ODBC Driver 17 for SQL Server};"
        "Server=localhost;"
        "Database=VALUO;"
        "Trusted_Connection=yes;"
    )
    connection_url = f"mssql+pyodbc:///?odbc_connect={params_conn}"
    engine = create_engine(connection_url)
    
    # Načtení vstupních dat z tabulky Valuo_data, kde nemovitost = 'byt'
    query = """
	SELECT id AS id_valuo, kat_uzemi, adresa, popis
    FROM Valuo_data 
    WHERE 1=1 
  	  and typ = 'byt'
      AND adresa <> 'Neznámá adresa'
      AND KN_WFS_info IS NULL
	  --AND KN_WFS_info =0
	  --and id = 1369

    """
    df_input = pd.read_sql(query, engine)
    logger.info("\nVstupní data z DB:")
    logger.info(df_input.head())
    
    total_db_records = len(df_input)
    queries_attempted = 0
    successful_queries = 0
    unsuccessful_ids = []  # seznam id_valuo, pro které dotaz nenašel parcelu
    
    df_list = []
    
    for idx, row in df_input.iterrows():
        id_valuo = row["id_valuo"]
        adresa = row["adresa"].strip()
        popis = row["popis"].strip() if row["popis"] is not None else ""
        if not adresa:
            continue
        
        # Pokud pole 'popis' obsahuje výraz "na parcele", použijeme novou metodu parsování
        if "na parcele" in popis.lower():
            parcels = extract_parcels_from_popis(popis, default_kat_uzemi=row["kat_uzemi"])
            if not parcels:
                logger.error(f"Nelze parsovat parcelní čísla z popisu: {popis}. Přeskakuji id_valuo={id_valuo}.")
                continue
            # Pro každý nalezený pár (kat_uzemi, parcelní číslo) provedeme dotaz
            for (kat, parc) in parcels:
                queries_attempted += 1
                logger.info(f"\nZpracovávám záznam id_valuo={id_valuo}, adresa='{adresa}', parcelní info: {kat}, {parc}")
                try:
                    df_parcel = get_parcel_data(kat, parc)
                    if df_parcel is not None and not df_parcel.empty:
                        successful_queries += 1
                        df_parcel["id_valuo"] = id_valuo  # propojení s Valuo_data
                        df_list.append(df_parcel)
                    else:
                        logger.info(f"Nebyly nalezeny data o parcele pro id_valuo={id_valuo} a parcelu {parc} ({kat}).")
                        unsuccessful_ids.append(id_valuo)
                except Exception as e:
                    logger.error(f"Chyba při zpracování id_valuo={id_valuo}, parcela {parc}: {e}")
                    unsuccessful_ids.append(id_valuo)
        else:
            # Původní logika pro případy, kdy se nejedná o nové zdrojové údaje
            try:
                if popis and "stavební" in popis.lower():
                    kat_uzemi, parc_cislo = parse_address(adresa, popis)
                else:
                    kat_uzemi, parc_cislo = parse_address(adresa, "")
                queries_attempted += 1
                logger.info(f"\nZpracovávám záznam id_valuo={id_valuo}, adresa='{adresa}'")
                df_parcel = get_parcel_data(kat_uzemi, parc_cislo)
                if df_parcel is not None and not df_parcel.empty:
                    successful_queries += 1
                    df_parcel["id_valuo"] = id_valuo
                    df_list.append(df_parcel)
                else:
                    logger.info(f"Nebyly nalezeny data o parcele pro id_valuo={id_valuo}.")
                    unsuccessful_ids.append(id_valuo)
            except Exception as e:
                logger.error(f"Chyba při zpracování id_valuo={id_valuo}: {e}")
                unsuccessful_ids.append(id_valuo)
    
    if df_list:
        df_all = pd.concat(df_list, ignore_index=True)
        logger.info("\nKonečný DataFrame se všemi získanými daty:")
        logger.info(df_all)
        
        # Vložení dat do tabulky KN_parcel_data
        df_all.to_sql("KN_parcel_data", con=engine, if_exists="append", index=False)
        inserted_rows = len(df_all)
        logger.info(f"\nData byla vložena do tabulky KN_parcel_data, počet vložených řádků: {inserted_rows}.")
    else:
        df_all = pd.DataFrame()
        logger.info("Nebyla získána žádná data o parcelách.")
    
    # Aktualizace tabulky Valuo_data:
    # Pro záznamy, u kterých byly získány data, nastavíme KN_WFS_info = 1,
    # a pro ty, kde nebyly data nalezeny, nastavíme KN_WFS_info = 0.
    successful_ids = df_all["id_valuo"].unique().tolist() if not df_all.empty else []
    # Předpokládáme, že všechny záznamy, které byly načteny z DB, jsou v df_input.
    all_ids = df_input["id_valuo"].unique().tolist()
    #all_ids = df_input["id_valuo"].iloc[:, 0].unique().tolist()

    unsuccessful_ids = list(set(all_ids) - set(successful_ids))
    
    # Aktualizace úspěšných záznamů: KN_WFS_info = 1
    updated_success = 0
    if successful_ids:
        id_str_success = ",".join(str(i) for i in successful_ids)
        update_query_success = f"UPDATE Valuo_data SET KN_WFS_info = 1 WHERE id IN ({id_str_success})"
        with engine.begin() as conn:
            result_success = conn.execute(text(update_query_success))
            updated_success = result_success.rowcount if result_success.rowcount is not None else len(successful_ids)
        logger.info(f"\nTabulka Valuo_data aktualizována: pro záznamy s id IN ({id_str_success}) bylo nastaveno KN_WFS_info = 1.")
    
    # Aktualizace neúspěšných záznamů: KN_WFS_info = 0
    updated_unsuccess = 0
    if unsuccessful_ids:
        id_str_unsuccess = ",".join(str(i) for i in unsuccessful_ids)
        update_query_unsuccess = f"UPDATE Valuo_data SET KN_WFS_info = 0 WHERE id IN ({id_str_unsuccess})"
        with engine.begin() as conn:
            result_unsuccess = conn.execute(text(update_query_unsuccess))
            updated_unsuccess = result_unsuccess.rowcount if result_unsuccess.rowcount is not None else len(unsuccessful_ids)
        logger.info(f"\nTabulka Valuo_data aktualizována: pro záznamy s id IN ({id_str_unsuccess}) bylo nastaveno KN_WFS_info = 0.")
    
    # Souhrnné statistiky
    summary = "\n" + "="*30 + " SOUHRN STATISTIK " + "="*30 + "\n"
    summary += f"Celkový počet načtených záznamů z DB Valuo_data: {len(all_ids)}\n"
    summary += f"Počet dotazů odeslaných do katastru: {queries_attempted}\n"
    summary += f"Počet úspěšně zodpovězených dotazů (parcely získány): {successful_queries}\n"
    summary += f"Celkový počet nových řádků vložených do KN_parcel_data: {inserted_rows}\n"
    summary += f"Počet aktualizovaných řádků v tabulce Valuo_data (KN_WFS_info=1): {updated_success}\n"
    summary += f"Počet aktualizovaných řádků v tabulce Valuo_data (KN_WFS_info=0): {updated_unsuccess}\n"
    summary += "="*80 + "\n"
    logger.info(summary)
    print(summary)


2025-05-29 08:36:35,872 - INFO - 
Vstupní data z DB:
2025-05-29 08:36:35,872 - INFO - 
Vstupní data z DB:
2025-05-29 08:36:35,874 - INFO - Empty DataFrame
Columns: [id_valuo, kat_uzemi, adresa, popis]
Index: []
2025-05-29 08:36:35,874 - INFO - Empty DataFrame
Columns: [id_valuo, kat_uzemi, adresa, popis]
Index: []
2025-05-29 08:36:35,876 - INFO - Nebyla získána žádná data o parcelách.
2025-05-29 08:36:35,876 - INFO - Nebyla získána žádná data o parcelách.


NameError: name 'inserted_rows' is not defined

In [1]:
#  *** 3 *** SQL query pro tahani PARCEL a BYTU k vizualizaci
query_pozemky = """
        -- CTE pro výběr pouze kompletních a bezchybových parcelních záznamů
        WITH ValidValuo AS (
            SELECT *
            FROM dbo.Valuo_data
            WHERE cislo_vkladu IN (
                -- Podmínka: všechny řádky pro dané cislo_vkladu jsou typu 'parcela'
                -- a žádná nemá chybu GPS (GPS_API_info = 'ERR')
                SELECT cislo_vkladu
                FROM dbo.Valuo_data
                GROUP BY cislo_vkladu
                HAVING 
                    COUNT(*) = COUNT(CASE WHEN nemovitost = 'parcela' THEN 1 END)  -- všechny záznamy jsou parcely
                    AND COUNT(CASE WHEN GPS_API_info = 'ERR' THEN 1 END) = 0       -- žádný záznam nemá GPS chybu
            )
        ),
        -- CTE pro spočítání počtu DISTINCT adres (parcel) na vložení
        ParcelCounts AS (
            SELECT 
                V.cislo_vkladu,
                COUNT(DISTINCT V.adresa) AS ParcelCount   -- kolik unikátních adres/parcel je k cislo_vkladu
            FROM dbo.Valuo_data V
            LEFT JOIN dbo.KN_parcel_data K
                ON K.id_valuo = V.id
            GROUP BY V.cislo_vkladu
        )

        -- Hlavní dotaz sestavující finální výsledky
        SELECT 
            V.*,
            --K.kat_uzemi,
            K.upper_zoning_id,
            K.parcel_number,
            K.gml_id,
            K.areaValue_m2,
            K.beginLifespanVersion,
            K.endLifespanVersion,
            K.geometry,
            K.inspire_localId,
            K.inspire_namespace,
            K.label,
            K.nationalCadastralReference,
            K.refPoint_x,
            K.refPoint_y,
            K.refPoint_lon,
            K.refPoint_lat,
            K.validFrom,
            K.administrativeUnit_href,
            K.administrativeUnit_title,
            K.zoning_href,
            K.zoning_title,
            K.id_valuo,
            -- Výpočet jednotkové ceny (JC):
            -- cenovy_udaj děleno součtem plochy všech parcel pro dané cislo_vkladu,
            -- dělení chráněno proti dělení nulou, výsledek zaokrouhlen na 0 desetinných míst,
            -- převeden na DECIMAL(38,0)
            CAST(
                ROUND(
                    V.cenovy_udaj 
                    / NULLIF(
                        SUM(V.plocha) OVER (PARTITION BY V.cislo_vkladu), 
                        0
                    ), 
                    0
                ) 
                AS DECIMAL(38,0)
            ) AS JC,
            -- Počet parcel pro dané cislo_vkladu
            PC.ParcelCount AS [#PARCEL]
        FROM ValidValuo AS V
        -- Připojení dat katastrálního operátu
        LEFT JOIN dbo.KN_parcel_data AS K
            ON K.id_valuo = V.id
        -- Připojení počtu parcel
        LEFT JOIN ParcelCounts AS PC
            ON PC.cislo_vkladu = V.cislo_vkladu
        WHERE 1 = 1
            -- Příklad filtru na katastrální území:
            -- AND V.kat_uzemi IN ('Škvorec', 'Úvaly u Prahy', 'Květnice')
            -- AND V.kat_uzemi IN ('Škvorec')
            -- Příklad filtru na konkrétní číslo vkladu:
            -- AND V.cislo_vkladu = 'V-2479/2024-101';

    """



query_byty = """
        WITH ValidValuo AS (
            SELECT *
            FROM dbo.Valuo_data
            WHERE cislo_vkladu IN (
                SELECT cislo_vkladu
                FROM dbo.Valuo_data
                GROUP BY cislo_vkladu
                HAVING COUNT(*) = 1
                AND MAX(typ) = 'byt'
            )
        ),
        JC_filtr AS (
            SELECT 
                V.*,  -- všechny sloupce z tabulky V
                -- Z tabulky K vybereme sloupce, které nebudou kolidovat s těmi z V, nebo jim dáme alias
                K.upper_zoning_id,
                K.parcel_number,
                K.gml_id,
                K.areaValue_m2,
                K.beginLifespanVersion,
                K.endLifespanVersion,
                K.geometry,
                K.inspire_localId,
                K.inspire_namespace,
                K.label,
                K.nationalCadastralReference,
                K.refPoint_x,
                K.refPoint_y,
                K.refPoint_lon,
                K.refPoint_lat,
                K.validFrom,
                K.administrativeUnit_href,
                K.administrativeUnit_title,
                K.zoning_href,
                K.zoning_title,
                K.id_valuo,
                K.kat_uzemi AS kn_kat_uzemi,  -- přejmenování duplicitního sloupce
                CAST(
                    ROUND(V.cenovy_udaj / NULLIF(SUM(V.plocha) OVER (PARTITION BY V.cislo_vkladu), 0), 0)
                    AS DECIMAL(38,0)
                ) AS JC
            FROM ValidValuo AS V
            LEFT JOIN dbo.KN_parcel_data AS K ON K.id_valuo = V.id
            WHERE V.plocha <> 0
            AND V.cenovy_udaj <> 0
            --AND V.kat_uzemi = 'Škvorec'
        )
        SELECT *
        FROM JC_filtr
        --WHERE JC >= 69600;

            ;



"""

In [ ]:
# *** 4 *** SPOJENÍ MAPOVÝCH PODKLADŮ - VALUO + KATASTR + ÚZEMNÍ PLÁN PRAHY ******************** VRSTVY - POZEMKY, BYTY ********* UPRAVA POPUPU ****** FUNKCNI


import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon

import folium
from folium import Map
from folium.elements import MacroElement
from folium.raster_layers import WmsTileLayer

from typing import Union

import logging
import urllib.parse
from sqlalchemy import create_engine

from jinja2 import Template
import webbrowser

from shapely.geometry import Polygon
from shapely.errors import TopologicalError

# ==============================================================================
# 1) Logging
# ==============================================================================
logger = logging.getLogger("ParcelViz")
logger.setLevel(logging.DEBUG)
fmt = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
ch = logging.StreamHandler();   ch.setLevel(logging.INFO);  ch.setFormatter(fmt);  logger.addHandler(ch)
fh = logging.FileHandler("process_log.txt", encoding="utf-8"); fh.setLevel(logging.DEBUG); fh.setFormatter(fmt); logger.addHandler(fh)

# ==============================================================================
# 2) Konstanty
# ==============================================================================
EXCHANGE_RATE_EUR_CZK = 25.0

# ==============================================================================
# 3) Pomocné funkce
# ==============================================================================

""" def parse_geometry(geom_str):
    try:
        coords = list(map(float, geom_str.split()))
        return Polygon([(coords[i], coords[i+1]) for i in range(0, len(coords), 2)])
    except Exception as e:
        logger.error(f"Chyba při převodu geometrie: {e}")
        return None """

def parse_geometry(geom_str):
    """
    Parsuje řetězec souřadnic na shapely.geometry.Polygon.
    - Pokud je vstup None/NaN nebo se objeví jakákoliv chyba,
      zaloguje ji a vrátí None.
    """
    # pokud je geom_str None (nebo v pandas NaN), rovnou skončíme
    if geom_str is None:
        logger.debug("Empty geometry string, returning None")
        return None

    try:
        coords = list(map(float, geom_str.split()))
        poly = Polygon([(coords[i], coords[i+1]) 
                        for i in range(0, len(coords), 2)])
        return poly

    except ValueError as e:
        logger.error(f"Non-numeric coordinate in geometry '{geom_str}': {e}")
        return None

    except IndexError as e:
        logger.error(
            f"Odd number of coordinate values in '{geom_str}', "
            "expected pairs of (x, y)."
        )
        return None

    except TopologicalError as e:
        logger.error(f"Invalid polygon topology for '{geom_str}': {e}")
        return None

    except Exception as e:
        # poslední pojistka pro neočekávané chyby
        logger.error(f"Chyba při převodu geometrie '{geom_str}': {e}")
        return None

#def format_thousand_space(val):
#    try:
#        return f"{float(val):,.0f}".replace(",", " ")
#    except:
#        return str(val)


def format_thousand_space(val, decimals: int = 2) -> str:
    """
    Formátuje číslo s mezerami jako oddělovači tisíců
    a čárkou jako desetinou čárkou.

    - Pro celočíselné hodnoty (např. 10560.0) vrátí "10 560"
    - Pro hodnoty s desetinnou částí (např. 56.39) vrátí "56,39"
      (vždy s přesně `decimals` místy za čárkou)
    """
    try:
        fval = float(val)
    except (ValueError, TypeError):
        # pokud se to nepovede převést na float, vrať původní řetězec
        return str(val)

    # celočíselné
    if fval.is_integer():
        # int(fval) odstraní .0 a f"{:,}" vloží čárky pro tisíce
        s = f"{int(fval):,}"
        # čárky jako tisícové oddělovače zaměníme za mezery
        return s.replace(",", " ")

    # má desetinnou část → formátujeme s pevnými `decimals` místy
    # f"{fval:,.2f}" => "1,234.56" nebo "56.39"
    s = f"{fval:,.{decimals}f}"
    # čárky pro tisíce → mezery, tečku desetinnou → čárka
    return s.replace(",", " ").replace(".", ",")


# ========================================================================
# Upravená funkce pro pozemky (vraťte sem původní dlaždice a stylování)
# ========================================================================
def aggregate_popup_pozemky(g):
    first = g.iloc[0]
    entries = []
    jc_values = []

    # vezmeme všechna unikátní čísla vkladu (bez .head(4))
    unique_entries = g.drop_duplicates(subset=['cislo_vkladu'])
    for _, row in unique_entries.iterrows():
        datum = format_datum_podani(row['datum_podani'])
        plocha = format_thousand_space(row['plocha'])
        cu = float(row.get('cenovy_udaj') or 0)
        jc = float(row.get('JC') or 0)
        if row.get('mena')=='EUR':
            cu_czk = cu * EXCHANGE_RATE_EUR_CZK
            jc_czk = jc * EXCHANGE_RATE_EUR_CZK
            cen = f"{format_thousand_space(cu)} EUR / {format_thousand_space(cu_czk)} CZK"
            jc_str = f"{format_thousand_space(jc_czk)} CZK/m²"
        else:
            cu_czk, jc_czk = cu, jc
            cen = f"{format_thousand_space(cu_czk)} CZK"
            jc_str = f"{format_thousand_space(jc_czk)} CZK/m²"

        jc_values.append(jc_czk)

        entries.append(f"""
            <table style="font-size:10px; margin-bottom:8px;">
              <hr style="border-top:2px solid #007bff; margin:4px 8px;">
              <tr><td><strong>Č. vkladu:</strong></td><td>{row['cislo_vkladu']}</td></tr>
              <tr><td><strong>Počet parcel:</strong></td><td>{row['#PARCEL']}</td></tr>
              <tr><td><strong>Datum podání:</strong></td><td>{datum}</td></tr>
              <tr><td><strong>Okres:</strong></td><td>{row['okres']}</td></tr>
              <tr><td><strong>K.území:</strong></td><td>{row['kat_uzemi']}</td></tr>
              <tr><td><strong>Kód k.území:</strong></td><td>{row['upper_zoning_id']}</td></tr>
              <tr><td><strong>Typ:</strong></td><td>{row['typ']}</td></tr>
              <tr><td><strong>Parc.č.:</strong></td><td>{row['parcel_number']}</td></tr>
              <tr><td><strong>Plocha:</strong></td><td>{plocha} m²</td></tr>
              <tr><td><strong>Cenový údaj:</strong></td><td>{cen}</td></tr>
              <tr><td><strong>JC:</strong></td><td><b>{jc_str}<b></td></tr>
            </table>
        """)

    # vše zabalíme do scrollovacího DIVu
    scrollable = (
        '<div style="max-height:300px; overflow-y:auto; padding-right:4px;">'
        + "".join(entries) +
        '</div>'
    )

    # průměrná JC a počet
    avg_html = ""
    if jc_values:
        avg = sum(jc_values)/len(jc_values)
        avg_html = (
            f"<p><strong>Průměrná JC:</strong> {format_thousand_space(avg)} CZK/m²</p>"
        )
    count = f"<p><strong>Počet unikátních č.vkladu:</strong> {len(unique_entries)}</p>"

    # odkaz (můžete upravit parametry dle potřeby)
    parcel = urllib.parse.quote_plus(first['parcel_number'])
    ku     = urllib.parse.quote_plus(str(first['upper_zoning_id']))
    link = (
        "https://nahlizenidokn.cuzk.cz/VyberParcelu/Parcela/InformaceO"
        f"?cisloParcelni={parcel}&katastralniUzemi={ku}"
    )
    link_html = f'<p><a href="{link}" target="_blank">Otevřít hledání na katastru</a></p>'

    return scrollable + avg_html + count + link_html



# ========================================================================
# Upravená funkce pro byty
# ========================================================================
def aggregate_popup_byty(g):
    first = g.iloc[0]
    entries = []
    jc_values = []

    # Sestavíme seznam všech záznamů
    computed = []
    for cislo_vkladu, subdf in g.groupby('cislo_vkladu'):
        #plocha_avg = subdf['plocha'].astype(float).mean(skipna=True) or 0.0
        plocha_avg = (round(v, 2) if not pd.isna(v := subdf['plocha'].astype(float).mean(skipna=True)) else 0.0)
        mena = subdf.iloc[0].get('mena', 'CZK')
        cu_avg = subdf['cenovy_udaj'].astype(float).mean(skipna=True) or 0.0

        if mena == 'EUR':
            cu_czk = cu_avg * EXCHANGE_RATE_EUR_CZK
            cenovy_val = (
                f"{format_thousand_space(cu_avg)} EUR / "
                f"{format_thousand_space(cu_czk)} CZK"
            )
        else:
            cu_czk = cu_avg
            cenovy_val = f"{format_thousand_space(cu_czk)} CZK"

        jc_calc = (cu_czk / plocha_avg) if plocha_avg else 0.0
        jc_values.append(jc_calc)
        computed.append((cislo_vkladu, jc_calc, subdf.iloc[0], cenovy_val, plocha_avg))

    # Vygenerujeme HTML pro každý záznam
    for cislo_vkladu, jc_calc, row0, cenovy_val, plocha_avg in computed:
        datum = format_datum_podani(row0['datum_podani'])
        okres = row0.get('okres', '')
        kat_u = row0.get('kat_uzemi', '')
        adresa = row0.get('adresa', '')
        typ = row0.get('typ', '')
        popis = row0.get('popis', '')
        plocha_s = format_thousand_space(plocha_avg)
        jc_s = f"{format_thousand_space(jc_calc, decimals=0)} CZK/m²"

        entries.append(f"""
            <table style="font-size:10px; margin-bottom:8px;">
              <hr style="border-top:2px solid #007bff; margin:4px 8px;">
              <tr><td><strong>Číslo vkladu:</strong></td><td>{cislo_vkladu}</td></tr>
              <tr><td><strong>Datum podání:</strong></td><td>{datum}</td></tr>
              <tr><td><strong>Okres:</strong></td><td>{okres}</td></tr>
              <tr><td><strong>Katastrální území:</strong></td><td>{kat_u}</td></tr>
              <tr><td><strong>Adresa:</strong></td><td>{adresa}</td></tr>
              <tr><td><strong>Typ:</strong></td><td>{typ}</td></tr>
              <tr><td><strong>Plocha (prům.):</strong></td><td>{plocha_s} m²</td></tr>
              <tr><td><strong>Cenový údaj (prům.):</strong></td><td>{cenovy_val}</td></tr>
              <tr><td><strong>JC:</strong></td><td><b>{jc_s}<b></td></tr>
              <tr><td><strong>Popis:</strong></td>
                  <td style="max-width:150px; word-wrap:break-word;">{popis}</td></tr>
            </table>
        """)

    # Scrollovatelný kontejner
    scroll_html = (
        '<div style="max-height:300px; overflow-y:auto; padding-right:4px;">'
        + "".join(entries) +
        '</div>'
    )

    # Průměrná JC
    avg_html = ""
    if jc_values:
        avg = sum(jc_values) / len(jc_values)
        avg_html = f"<p><strong>Průměrná JC (v rámci budovy / parcely):</strong> {format_thousand_space(avg)} CZK/m²</p>"

    # Celkový počet
    total = len(computed)
    count_html = f"<p><strong>Celkem unikátních čísel vkladu:</strong> {total}</p>"

    # Odkaz do katastru
    parcel = urllib.parse.quote_plus(first['parcel_number'])
    ku     = urllib.parse.quote_plus(str(first['upper_zoning_id']))
    link_html = (
        f'<p><a href="https://nahlizenidokn.cuzk.cz/VyberParcelu/Parcela/InformaceO'
        f'?cisloParcelni={parcel}&katastralniUzemi={ku}" target="_blank">'
        "Otevřít hledání na katastru</a></p>"
    )

    return scroll_html + avg_html + count_html + link_html



# ==============================================================================
# 5) Připojení k databázi
# ==============================================================================
conn = urllib.parse.quote_plus(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=localhost;Database=VALUO;Trusted_Connection=yes;"
)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={conn}")

# ==============================================================================
# 6) Načtení a příprava dat – POZEMKY
# ==============================================================================
df_p = pd.read_sql(query_pozemky, engine)
df_p['geometry'] = df_p['geometry'].apply(parse_geometry)
gdf_p = gpd.GeoDataFrame(df_p, geometry='geometry', crs="EPSG:5514").to_crs(epsg=4326)

popup_p = (df_p
           .groupby('parcel_number')
           .apply(aggregate_popup_pozemky)
           .reset_index(name='popup_html'))

gdf_p_agg = (gdf_p
             .dissolve(by='parcel_number', as_index=False)
             .merge(popup_p, on='parcel_number', how='left'))
# zachováme jen geometry, JC a popup_html
gdf_p_agg = gdf_p_agg[['geometry','JC','popup_html']]

# ==============================================================================
# 7) Načtení a příprava dat – BYTY
# ==============================================================================
df_b = pd.read_sql(query_byty, engine)
df_b['geometry'] = df_b['geometry'].apply(parse_geometry)
gdf_b = gpd.GeoDataFrame(df_b, geometry='geometry', crs="EPSG:5514").to_crs(epsg=4326)

popup_b = (df_b
           .groupby('parcel_number')
           .apply(aggregate_popup_byty)
           .reset_index(name='popup_html'))

gdf_b_agg = (gdf_b
             .dissolve(by='parcel_number', as_index=False)
             .merge(popup_b, on='parcel_number', how='left'))
gdf_b_agg = gdf_b_agg[['geometry','JC','popup_html']]

# ==============================================================================
# 8) Funkce pro barvy
# ==============================================================================
def get_color_pozemky(jc):
    # vaše původní škála
    if jc <= 10:      return "#001200"
    elif jc <= 99:    return "#014701"
    elif jc <= 249:   return "#2B7203"
    elif jc <= 499:   return "#4CC705"
    elif jc <= 999:   return "#82DA06"
    elif jc <= 2999:  return "#B5D00A"
    elif jc <= 4999:  return "#E6DB0C"
    elif jc <= 9999:  return "#E8C30A"
    elif jc <= 14999: return "#E3B008"
    elif jc <= 19999: return "#EF9E06"
    elif jc <= 29999: return "#AC6207"
    elif jc <= 39999: return "#D35F07"
    elif jc <= 59999: return "#C32E04"
    else:             return "#F11909"

def get_color_byty(jc):
    # vaše původní škála
    if jc <= 24999:   return "#089208"
    elif jc <= 49999: return "#4CC705"
    elif jc <= 74999: return "#82DA06"
    elif jc <= 99999: return "#B5D00A"
    elif jc <= 119999:return "#E6DB0C"
    elif jc <= 129999:return "#E8C30A"
    elif jc <= 149999:return "#E3B008"
    elif jc <= 159999:return "#EF9E06"
    elif jc <= 169999:return "#AC6207"
    elif jc <= 179999:return "#D35F07"
    elif jc <= 189999:return "#C32E04"
    else:             return "#F11909"

def style_poz(feature):
    jc = float(feature['properties'].get('JC') or 0)
    return {'color':'black','weight':2,'fillColor':get_color_pozemky(jc),'fillOpacity':0.5}

def style_byt(feature):
    jc = float(feature['properties'].get('JC') or 0)
    return {'color':'black','weight':2,'fillColor':get_color_byty(jc),'fillOpacity':0.5}

# ==============================================================================
# 9) Vytvoření mapy
# ==============================================================================
center = [
    gdf_p_agg.geometry.centroid.y.mean(),
    gdf_p_agg.geometry.centroid.x.mean()
]
m = folium.Map(location=center, zoom_start=15, tiles=None)

# základní dlaždice
# Přidání defaultních TileLayerů Folia pro přepínání
# základní OSM

folium.TileLayer(
    tiles='OpenStreetMap',
    name='Open Street Map',
    attr='© OpenStreetMap contributors',
    show=True       # tato bude výchozí
).add_to(m)

# Turisticka mapa
folium.TileLayer(
    tiles='https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png',
    name='Open Topo Map',
    attr='© OpenTopoMap (CC-BY-SA)',
    overlay=False,    # základní dlaždice
    show=False
).add_to(m)


# CartoDB Positron
folium.TileLayer(
    tiles='https://cartodb-basemaps-a.global.ssl.fastly.net/light_all/{z}/{x}/{y}.png',
    name='CartoDB LIGHT',
    attr='© OpenStreetMap contributors © CARTO',
    show=False      # bude vypnutá
).add_to(m)

# CartoDB Dark Matter
folium.TileLayer(
    tiles='https://cartodb-basemaps-a.global.ssl.fastly.net/dark_all/{z}/{x}/{y}.png',
    name='CartoDB DARK',
    attr='© OpenStreetMap contributors © CARTO',
    show=False      # bude vypnutá
).add_to(m)



folium.TileLayer(
    tiles=(
        'https://ags.cuzk.gov.cz/arcgis1/rest/services/'
        'ORTOFOTO_WM/MapServer/tile/{z}/{y}/{x}'
    ),
    name='Ortofotomapa ČÚZK',
    attr='© ČÚZK',
    overlay=False,
    control=True,
    show=False  # změňte na True, pokud chcete vrstvu zapnout hned
).add_to(m)


# Ortofoto
folium.TileLayer(
    tiles='http://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    name='Ortofotomapa ESRI',
    attr='Esri World Imagery',
    overlay=False,
    control=True,
    show=False      # bude vypnutá
).add_to(m)

# Hranice parcel se budou nabízet až od zoomu 15 do 19
WmsTileLayer(
    url='https://services.cuzk.gov.cz/wms/local-km-wms.asp?',
    name='Hranice parcel',
    layers='hranice_parcel',
    fmt='image/png',
    transparent=True,
    version='1.3.0',
    attr='© ČÚZK',
    overlay=True,
    control=True,
    show=False,
    minZoom=0,
    maxZoom=30
).add_to(m)

# Parcelní čísla od zoomu 16 (až budou dost čitelné)
WmsTileLayer(
    url='https://services.cuzk.gov.cz/wms/local-km-wms.asp?',
    name='Parcelní čísla',
    layers='parcelni_cisla',
    fmt='image/png',
    transparent=True,
    version='1.3.0',
    attr='© ČÚZK',
    overlay=True,
    control=True,
    show=False,
    minZoom=0,
    maxZoom=30
).add_to(m)

# Základní Katastrální mapa (KN) necháme viditelnou od zoomu 0
WmsTileLayer(
    url='https://services.cuzk.gov.cz/wms/local-km-wms.asp?',
    name='Katastrální mapa (KN)',
    layers='KN',
    fmt='image/png',
    transparent=True,
    version='1.3.0',
    attr='© ČÚZK',
    overlay=True,
    control=True,
    show=False,
    minZoom=0,
    maxZoom=30
).add_to(m)


# Dynamická vrstva ÚP
class DynamicArcGISTileLayer(MacroElement):
    _template = Template(u"""
        {% macro script(this, kwargs) %}
            var DynamicLayer = L.TileLayer.extend({
                getTileUrl: function(coords) {
                    var tileSize = 256;
                    var initialResolution = 2 * Math.PI * 6378137 / tileSize;
                    var originShift = 2 * Math.PI * 6378137 / 2.0;
                    var resolution = initialResolution / Math.pow(2, coords.z);
                    var minx = coords.x * tileSize * resolution - originShift;
                    var maxx = (coords.x + 1) * tileSize * resolution - originShift;
                    var miny = originShift - (coords.y + 1) * tileSize * resolution;
                    var maxy = originShift - coords.y * tileSize * resolution;
                    var bbox = [minx, miny, maxx, maxy].join(",");
                    return "{{ this.url }}?bbox=" + bbox +
                           "&bboxSR=102100&imageSR=102100&size=256,256" +
                           "&format=png32&transparent=true&layers=show:0&f=image";
                }
            });
            var dynamicLayer = new DynamicLayer();
            dynamicLayer.addTo({{ this._parent.get_name() }});
        {% endmacro %}
    """)
    def __init__(self, url):
        super().__init__()
        self._name='DynamicArcGISTileLayer'
        self.url = url

arcgis_url = "https://gs-pub.praha.eu/arcgis/rest/services/pup/uzemni_plan_platny/MapServer/export"
dynamic_fg = folium.FeatureGroup(
    name="Územní plán Prahy – plán využití",
    overlay=True, control=True, show=False      # bude vypnutá
)
dynamic_fg.add_child(DynamicArcGISTileLayer(arcgis_url))
m.add_child(dynamic_fg)

# BYTY
popup_b_layer = folium.features.GeoJsonPopup(fields=['popup_html'],
                                             labels=False,
                                             parse_html=True,
                                             max_width=400)
folium.GeoJson(
    gdf_b_agg.to_json(),
    name="BYTY",
    style_function=style_byt,
    popup=popup_b_layer
).add_to(m)

# POZEMKY
popup_p_layer = folium.features.GeoJsonPopup(fields=['popup_html'],
                                             labels=False,
                                             parse_html=True,
                                             max_width=400)
folium.GeoJson(
    gdf_p_agg.to_json(),
    name="POZEMKY",
    style_function=style_poz,
    popup=popup_p_layer
).add_to(m)



folium.LayerControl().add_to(m)



# ==============================================================================
# 10) Uložit a otevřít
# ==============================================================================
out = "DATA_POZEMKY_BYTY.html"
m.save(out)
logger.info(f"Mapa uložena do '{out}'.")
webbrowser.open(out)

C:\Users\ijttr\AppData\Local\Temp\ipykernel_53700\3583506037.py:303: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(aggregate_popup_pozemky)
C:\Users\ijttr\AppData\Local\Temp\ipykernel_53700\3583506037.py:321: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(aggregate_popup_byty)
C:\Users\ijttr\AppData\Local\Temp\ipykernel_53700\3583506037.py:376: UserWarning: Geometry is in a geographic CRS. Resu

True

In [23]:
# ***SPOJENÍ MAPOVÝCH PODKLADŮ - VALUO + KATASTR + ÚZEMNÍ PLÁN PRAHY ******************** VRSTVY - POZEMKY, BYTY *********   FUNKCNI



import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import folium
import logging
import urllib.parse
from sqlalchemy import create_engine
from folium.elements import MacroElement
from jinja2 import Template
import webbrowser

# ==============================================================================
# Nastavení logování: výpis do konzole i do souboru "process_log.txt"
# ==============================================================================
logger = logging.getLogger("ParcelViz")
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
console_handler.setFormatter(formatter)
logger.addHandler(console_handler)

file_handler = logging.FileHandler("process_log.txt", encoding="utf-8")
file_handler.setLevel(logging.DEBUG)
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

# ==============================================================================
# Konstanta pro přepočet EUR -> CZK
# Upravte dle reálného kurzu nebo načítejte dynamicky
# ==============================================================================
EXCHANGE_RATE_EUR_CZK = 25.0

# ==============================================================================
# Funkce pro převod textového řetězce geometrie (souřadnicový řetězec) na shapely Polygon
# ==============================================================================
def parse_geometry(geom_str):
    """
    Předpokládá, že geom_str obsahuje souřadnice oddělené mezerou, např.:
      "x1 y1 x2 y2 ... xN yN"
    Vrací objekt shapely.geometry.Polygon.
    """
    try:
        coords = list(map(float, geom_str.split()))
        points = [(coords[i], coords[i+1]) for i in range(0, len(coords), 2)]
        return Polygon(points)
    except Exception as e:
        logger.error(f"Chyba při převodu geometrie: {e}")
        return None

# ==============================================================================
# Připojení k databázi VALUO pomocí SQLAlchemy
# ==============================================================================
params_conn = urllib.parse.quote_plus(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=localhost;"
    "Database=VALUO;"
    "Trusted_Connection=yes;"
)
connection_url = f"mssql+pyodbc:///?odbc_connect={params_conn}"
engine = create_engine(connection_url)

# ==============================================================================
# Pomocné funkce pro formátování
# ==============================================================================
def format_thousand_space(value):
    """
    Převede číselnou hodnotu na řetězec bez desetinných míst
    a s oddělením tisíců mezerou (např. 123456.789 -> '123 457').
    """
    try:
        return f"{float(value):,.0f}".replace(",", " ")
    except (ValueError, TypeError):
        return str(value)

def format_datum_podani(datum_str):
    """
    Převede datum 'YYYY-MM-DD HH:MM:SS.sss' na formát:
    MĚSÍC/ROK, (původní celý datum/časový údaj).
    Např. '2024-12-02 13:17:27.000' -> '<strong>12/2024</strong>, (2024-12-02 13:17:27.000)'
    Pokud parsování selže, vrátí původní řetězec.
    """
    try:
        dt_parsed = pd.to_datetime(datum_str)
        month_year = dt_parsed.strftime("%m/%Y")
        return f"<strong>{month_year}</strong>, ({datum_str})"
    except:
        return datum_str

# ==============================================================================
# --- Agregace tooltipu pro POZEMKY ---
# ==============================================================================
def aggregate_tooltip_text_formatted_pozemky(group):
    # Vybereme pouze první záznam pro každé unikátní cislo_vkladu
    unique_entries = group.drop_duplicates(subset=['cislo_vkladu'])
    tooltip_entries = []
    jc_values = []  # Pro výpočet průměrné JC

    # Procházíme maximálně 4 záznamy
    for idx, row in unique_entries.head(4).iterrows():
        datum_podani_val = format_datum_podani(row['datum_podani'])
        plocha_val = format_thousand_space(row['plocha'])
        try:
            cu_original = float(row['cenovy_udaj'])
        except (ValueError, TypeError):
            cu_original = 0.0
        try:
            jc_original = float(row['JC'])
        except (ValueError, TypeError):
            jc_original = 0.0

        mena = row['mena']
        if mena == 'EUR':
            cu_czk = cu_original * EXCHANGE_RATE_EUR_CZK
            jc_czk = jc_original * EXCHANGE_RATE_EUR_CZK
            cenovy_udaj_val = f"{format_thousand_space(cu_original)} EUR / {format_thousand_space(cu_czk)} CZK"
            jc_val_str = f"{format_thousand_space(jc_czk)} CZK/m²"
        else:
            cu_czk = cu_original
            jc_czk = jc_original
            cenovy_udaj_val = f"{format_thousand_space(cu_czk)} CZK"
            jc_val_str = f"{format_thousand_space(jc_czk)} CZK/m²"

        jc_values.append(jc_czk)

        entry = (
            f"<table style='font-size:10px;'>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Číslo vkladu:</td>"
            f"<td style='padding-left: 10px;'>{row['cislo_vkladu']}</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Počet parcel v rámci vkladu:</td>"
            f"<td style='padding-left: 10px;'>{row['#PARCEL']}</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Datum podání:</td>"
            f"<td style='padding-left: 10px;'>{datum_podani_val}</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Okres:</td>"
            f"<td style='padding-left: 10px;'>{row['okres']}</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Katastrální území:</td>"
            f"<td style='padding-left: 10px;'>{row['kat_uzemi']}</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Kód kat. území:</td>"
            f"<td style='padding-left: 10px;'>{row['upper_zoning_id']}</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Typ:</td>"
            f"<td style='padding-left: 10px;'>{row['typ']}</td></tr>"            
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Parc.č.:</td>"
            f"<td style='padding-left: 10px;'>{row['parcel_number']}</td></tr>"            
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Plocha:</td>"
            f"<td style='padding-left: 10px;'>{plocha_val} m²</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Cenový údaj:</td>"
            f"<td style='padding-left: 10px;'>{cenovy_udaj_val}</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>JC:</td>"
            f"<td style='padding-left: 10px;'>{jc_val_str}</td></tr>"
            f"</table>"
        )
        tooltip_entries.append(entry)

    aggregated_text = "<hr>".join(tooltip_entries)

    # Přidáme výpočet průměrné JC
    if len(jc_values) > 0:
        avg_jc_czk = sum(jc_values) / len(jc_values)
        avg_jc_str = format_thousand_space(avg_jc_czk)
        aggregated_text += (
            f"<br><table style='font-size:14px;'><tr>"
            f"<td style='font-weight:bold; vertical-align: top;'>Průměrná JC:</td>"
            f"<td style='padding-left: 10px;'>{avg_jc_str} CZK/m²</td>"
            f"</tr></table>"
        )

    unique_count = unique_entries.shape[0]
    aggregated_text += f"<br><span style='font-size:12px; font-weight:bold;'>Celkem unikátních čísel vkladu: {unique_count}</span>"
    return aggregated_text

# ==============================================================================
# --- Agregace tooltipu pro BYTY ---
# ==============================================================================
def aggregate_tooltip_text_formatted_byty_for_parcel(parcel_group):
    lines_for_cislo_vkladu = []
    jc_values = []  # Pro výpočet průměrné JC
    computed = []  # Uložíme výsledky pro každé unikátní cislo_vkladu

    grouped_by_cislo = parcel_group.groupby('cislo_vkladu')
    for cislo_vkladu, subdf in grouped_by_cislo:
        plocha_avg = subdf['plocha'].astype(float).mean(skipna=True) or 0.0
        mena = subdf.iloc[0].get('mena', 'CZK')
        cenovy_udaj_avg_original = subdf['cenovy_udaj'].astype(float).mean(skipna=True) or 0.0

        if mena == 'EUR':
            cenovy_udaj_avg_czk = cenovy_udaj_avg_original * EXCHANGE_RATE_EUR_CZK
            cenovy_udaj_val_for_tooltip = (
                f"{format_thousand_space(cenovy_udaj_avg_original)} EUR / "
                f"{format_thousand_space(cenovy_udaj_avg_czk)} CZK"
            )
        else:
            cenovy_udaj_avg_czk = cenovy_udaj_avg_original
            cenovy_udaj_val_for_tooltip = f"{format_thousand_space(cenovy_udaj_avg_czk)} CZK"

        if plocha_avg == 0:
            jc_calc = 0.0
        else:
            jc_calc = cenovy_udaj_avg_czk / plocha_avg

        jc_values.append(jc_calc)
        computed.append((cislo_vkladu, jc_calc, subdf.iloc[0], cenovy_udaj_val_for_tooltip, plocha_avg))

    # Vypíšeme pouze první 4 segmenty
    for i, (cislo_vkladu, jc_calc, row0, cenovy_udaj_val_for_tooltip, plocha_avg) in enumerate(computed):
        if i >= 4:
            break
        datum_podani_val = format_datum_podani(row0['datum_podani'])
        plocha_avg_str = format_thousand_space(plocha_avg)
        jc_calc_str = f"{format_thousand_space(jc_calc)} CZK/m²"
        okres = row0.get('okres', '')
        kat_uzemi = row0.get('kat_uzemi', '')
        adresa = row0.get('adresa', '')
        typ = row0.get('typ', '')
        popis = row0.get('popis', '')
        line_html = (
            f"<table style='font-size:10px;'>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Číslo vkladu:</td>"
            f"<td style='padding-left: 10px;'>{cislo_vkladu}</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Datum podání:</td>"
            f"<td style='padding-left: 10px;'>{datum_podani_val}</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Okres:</td>"
            f"<td style='padding-left: 10px;'>{okres}</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Katastrální území:</td>"
            f"<td style='padding-left: 10px;'>{kat_uzemi}</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Adresa:</td>"
            f"<td style='padding-left: 10px;'>{adresa}</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Typ:</td>"
            f"<td style='padding-left: 10px;'>{typ}</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Plocha (prům.):</td>"
            f"<td style='padding-left: 10px;'>{plocha_avg_str} m²</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Cenový údaj (prům.):</td>"
            f"<td style='padding-left: 10px;'>{cenovy_udaj_val_for_tooltip}</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>JC:</td>"
            f"<td style='padding-left: 10px;'>{jc_calc_str}</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Popis:</td>"
            f"<td style='padding-left: 10px; max-width:150px; word-wrap: break-word; white-space: normal;'>{popis}</td></tr>"
            f"</table>"
        )
        lines_for_cislo_vkladu.append(line_html)

    result_html = "<hr>".join(lines_for_cislo_vkladu)

    # Přidáme výpočet průměrné JC
    if len(jc_values) > 0:
        avg_jc_for_parcel = sum(jc_values) / len(jc_values)
        avg_jc_str = format_thousand_space(avg_jc_for_parcel)
        result_html += (
            f"<br><table style='font-size:14px;'><tr>"
            f"<td style='font-weight:bold; vertical-align: top;'>Průměrná JC (v rámci budovy / parcely):</td>"
            f"<td style='padding-left: 10px;'>{avg_jc_str} CZK/m²</td>"
            f"</tr></table>"
        )

    total_unique = len(computed)
    result_html += f"<br><span style='font-size:12px; font-weight:bold;'>Celkem unikátních čísel vkladu: {total_unique}</span>"
    return result_html

# ==============================================================================
# ---------------------- Data POZEMKY: Dotaz + úprava + vykreslení -------------
# ==============================================================================
df_pozemky = pd.read_sql(query_pozemky, engine)
logger.info("Načtených záznamů z KN_parcel_data:")
logger.info(df_pozemky.head())

df_pozemky = df_pozemky.loc[:, ~df_pozemky.columns.duplicated()]
df_pozemky['geom'] = df_pozemky['geometry'].apply(parse_geometry)

gdf_pozemky = gpd.GeoDataFrame(df_pozemky, geometry='geom', crs="EPSG:5514")
gdf_pozemky = gdf_pozemky.drop(columns=['geometry']).rename(columns={'geom': 'geometry'})
gdf_pozemky = gdf_pozemky.set_geometry("geometry").to_crs(epsg=4326)

tooltip_aggregated_poz = (
    df_pozemky
    .groupby('parcel_number')
    .apply(aggregate_tooltip_text_formatted_pozemky)
    .reset_index()
)
tooltip_aggregated_poz.columns = ['parcel_number', 'tooltip_text']

gdf_poz_agg = gdf_pozemky.dissolve(by='parcel_number', as_index=False)
gdf_poz_agg = gdf_poz_agg.merge(tooltip_aggregated_poz, on='parcel_number', how='left')

def get_color_pozemky(jc_value):
    if jc_value <= 10:
        return "#001200"   
    elif jc_value <= 99:
        return "#014701"   
    elif jc_value <= 249:
        return "#2B7203"
    elif jc_value <= 499:
        return "#4CC705"
    elif jc_value <= 999:
        return "#82DA06"
    elif jc_value <= 2999:
        return "#B5D00A"
    elif jc_value <= 4999:
        return "#E6DB0C"
    elif jc_value <= 9999:
        return "#E8C30A"
    elif jc_value <= 14999:
        return "#E3B008"
    elif jc_value <= 19999:
        return "#EF9E06"
    elif jc_value <= 29999:
        return "#AC6207"    
    elif jc_value <= 39999:
        return "#D35F07"
    elif jc_value <= 59999:
        return "#C32E04"
    else:
        return "#F11909"

def style_function_pozemky(feature):
    try:
        jc_value = float(feature["properties"]["JC"])
    except (KeyError, TypeError, ValueError):
        jc_value = 0
    return {
        'color': 'black',
        'weight': 2,
        'fillColor': get_color_pozemky(jc_value),
        'fillOpacity': 0.5,
    }

tooltip_pozemky = folium.GeoJsonTooltip(
    fields=['tooltip_text'],
    aliases=[''],
    localize=True,
    sticky=True,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 3px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=300,
)

gdf_poz_agg_json = gdf_poz_agg.to_json(default=str)

# ==============================================================================
# ---------------------- Data BYTY: Dotaz + úprava + vykreslení ----------------
# ==============================================================================
df_byty = pd.read_sql(query_byty, engine)
logger.info("Načtených záznamů z KN_bytu_data:")
logger.info(df_byty.head())

df_byty = df_byty.loc[:, ~df_byty.columns.duplicated()]
df_byty['geom'] = df_byty['geometry'].apply(parse_geometry)

gdf_byty = gpd.GeoDataFrame(df_byty, geometry='geom', crs="EPSG:5514")
gdf_byty = gdf_byty.drop(columns=['geometry']).rename(columns={'geom': 'geometry'})
gdf_byty = gdf_byty.set_geometry("geometry").to_crs(epsg=4326)

tooltip_aggregated_byty = (
    df_byty
    .groupby('parcel_number')
    .apply(aggregate_tooltip_text_formatted_byty_for_parcel)
    .reset_index()
)
tooltip_aggregated_byty.columns = ['parcel_number', 'tooltip_text']

gdf_byty_agg = gdf_byty.dissolve(by='parcel_number', as_index=False)
gdf_byty_agg = gdf_byty_agg.merge(tooltip_aggregated_byty, on='parcel_number', how='left')

def get_color_byty(jc_value):
    if jc_value <= 24999:
        return "#089208"   # tmavě zelená
    elif jc_value <= 49999:
        return "#4CC705"
    elif jc_value <= 74999:
        return "#82DA06"
    elif jc_value <= 99999:
        return "#B5D00A"
    elif jc_value <= 119999:
        return "#E6DB0C"
    elif jc_value <= 129999:
        return "#E8C30A"
    elif jc_value <= 149999:
        return "#E3B008"
    elif jc_value <= 159999:
        return "#EF9E06"
    elif jc_value <= 169999:
        return "#AC6207"    
    elif jc_value <= 179999:
        return "#D35F07"
    elif jc_value <= 189999:
        return "#C32E04"
    else:
        return "#F11909"

def style_function_byty(feature):
    try:
        jc_value = float(feature["properties"]["JC"])
    except (KeyError, TypeError, ValueError):
        jc_value = 0
    return {
        'color': 'black',
        'weight': 2,
        'fillColor': get_color_byty(jc_value),
        'fillOpacity': 0.5,
    }

tooltip_byty = folium.GeoJsonTooltip(
    fields=['tooltip_text'],
    aliases=[''],
    localize=True,
    sticky=True,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 3px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=300,
)

gdf_byty_agg_json = gdf_byty_agg.to_json(default=str)

# ==============================================================================
# ---------------------- Vytvoření interaktivní mapy ---------------------------
# ==============================================================================
# --- Vytvoření interaktivní mapy se všemi vestavěnými dlaždicemi pro přepínání ---
center_lat = gdf_poz_agg.geometry.centroid.y.mean()
center_lon = gdf_poz_agg.geometry.centroid.x.mean()


# Vytvoření interaktivní mapy s centrem na průměrné souřadnice pozemků
# m = folium.Map(location=[center_lat, center_lon], zoom_start=15)


# Základní mapa bez přednastavených dlaždic
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=15,
    tiles=None
)

# Přidání defaultních TileLayerů Folia pro přepínání
# základní OSM
folium.TileLayer(
    tiles='OpenStreetMap',
    name='OpenStreetMap',
    attr='© OpenStreetMap contributors'
).add_to(m)

# Stamen Terrain
folium.TileLayer(
    tiles='https://stamen-tiles.a.ssl.fastly.net/terrain/{z}/{x}/{y}.png',
    name='Stamen Terrain',
    attr='Map tiles by Stamen Design, CC BY 3.0 — Data © OpenStreetMap contributors'
).add_to(m)

# Stamen Toner
folium.TileLayer(
    tiles='https://stamen-tiles.a.ssl.fastly.net/toner/{z}/{x}/{y}.png',
    name='Stamen Toner',
    attr='Map tiles by Stamen Design, CC BY 3.0 — Data © OpenStreetMap contributors'
).add_to(m)

# CartoDB Positron
folium.TileLayer(
    tiles='https://cartodb-basemaps-a.global.ssl.fastly.net/light_all/{z}/{x}/{y}.png',
    name='CartoDB positron',
    attr='© OpenStreetMap contributors © CARTO'
).add_to(m)

# CartoDB Dark Matter
folium.TileLayer(
    tiles='https://cartodb-basemaps-a.global.ssl.fastly.net/dark_all/{z}/{x}/{y}.png',
    name='CartoDB dark_matter',
    attr='© OpenStreetMap contributors © CARTO'
).add_to(m)



# Vrstva BYTY
folium.GeoJson(
    gdf_byty_agg_json,
    name="BYTY",
    style_function=style_function_byty,
    tooltip=tooltip_byty
).add_to(m)

# Vrstva POZEMKY
folium.GeoJson(
    gdf_poz_agg_json,
    name="POZEMKY",
    style_function=style_function_pozemky,
    tooltip=tooltip_pozemky
).add_to(m)

# Ortofoto mapa
folium.TileLayer(
    tiles='http://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Esri World Imagery',
    name='Ortofoto Mapa',
    overlay=True,
    control=True
).add_to(m)

# Dynamická vrstva Územního plánu
class DynamicArcGISTileLayer(MacroElement):
    _template = Template(u"""
        {% macro script(this, kwargs) %}
            var DynamicLayer = L.TileLayer.extend({
                getTileUrl: function(coords) {
                    var tileSize = 256;
                    var initialResolution = 2 * Math.PI * 6378137 / tileSize;
                    var originShift = 2 * Math.PI * 6378137 / 2.0;
                    var resolution = initialResolution / Math.pow(2, coords.z);
                    var minx = coords.x * tileSize * resolution - originShift;
                    var maxx = (coords.x + 1) * tileSize * resolution - originShift;
                    var miny = originShift - (coords.y + 1) * tileSize * resolution;
                    var maxy = originShift - coords.y * tileSize * resolution;
                    var bbox = [minx, miny, maxx, maxy].join(",");
                    return "{{ this.url }}?bbox=" + bbox +
                           "&bboxSR=102100&imageSR=102100&size=256,256" +
                           "&format=png32&transparent=true&layers=show:0&f=image";
                }
            });
            var dynamicLayer = new DynamicLayer();
            dynamicLayer.addTo({{ this._parent.get_name() }});
        {% endmacro %}
    """)
    def __init__(self, url):
        super(DynamicArcGISTileLayer, self).__init__()
        self._name = 'DynamicArcGISTileLayer'
        self.url = url

arcgis_export_url = "https://gs-pub.praha.eu/arcgis/rest/services/pup/uzemni_plan_platny/MapServer/export"
dynamic_group = folium.FeatureGroup(
    name="Územní plán Hlavního města Prahy (výkres č. 4) – Plán využití ploch",
    overlay=True,
    control=True
)
dynamic_layer = DynamicArcGISTileLayer(arcgis_export_url)
dynamic_group.add_child(dynamic_layer)
m.add_child(dynamic_group)

folium.LayerControl().add_to(m)

output_file = "DATA_POZEMKY_BYTY.html"
m.save(output_file)
logger.info(f"Interaktivní mapa byla uložena do souboru '{output_file}'.")
webbrowser.open(output_file)


2025-05-28 13:43:14,299 - INFO - Načtených záznamů z KN_parcel_data:
2025-05-28 13:43:14,299 - INFO - Načtených záznamů z KN_parcel_data:
2025-05-28 13:43:14,299 - INFO - Načtených záznamů z KN_parcel_data:
2025-05-28 13:43:14,299 - INFO - Načtených záznamů z KN_parcel_data:
2025-05-28 13:43:14,299 - INFO - Načtených záznamů z KN_parcel_data:
2025-05-28 13:43:14,299 - INFO - Načtených záznamů z KN_parcel_data:
2025-05-28 13:43:14,299 - INFO - Načtených záznamů z KN_parcel_data:
2025-05-28 13:43:14,299 - INFO - Načtených záznamů z KN_parcel_data:
2025-05-28 13:43:14,299 - INFO - Načtených záznamů z KN_parcel_data:
2025-05-28 13:43:14,299 - INFO - Načtených záznamů z KN_parcel_data:
2025-05-28 13:43:14,306 - INFO -       id           timestamp      cislo_vkladu        datum_podani  \
0  18218 2025-02-03 19:18:12     V-10/2023-101 2023-01-02 08:58:57   
1   9248 2025-02-03 16:28:22  V-10027/2023-211 2023-12-21 11:54:17   
2   9249 2025-02-03 16:28:22  V-10027/2023-211 2023-12-21 11:54:17 

True

In [10]:
# Testovani uprav vystupu mapy - vykreslovani JC primo do mapy
# ***SPOJENÍ MAPOVÝCH PODKLADŮ - VALUO + KATASTR + ÚZEMNÍ PLÁN PRAHY ******************** VRSTVY - POZEMKY, BYTY -  POPISKY AVG JC PRIMO V MAPE *********   FUNKCNI



import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import folium
import logging
import urllib.parse
from sqlalchemy import create_engine
from folium.elements import MacroElement
from jinja2 import Template
import webbrowser

# ==============================================================================
# Nastavení logování: výpis do konzole i do souboru "process_log.txt"
# ==============================================================================
logger = logging.getLogger("ParcelViz")
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
console_handler.setFormatter(formatter)
logger.addHandler(console_handler)

file_handler = logging.FileHandler("process_log.txt", encoding="utf-8")
file_handler.setLevel(logging.DEBUG)
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

# ==============================================================================
# Konstanta pro přepočet EUR -> CZK
# Upravte dle reálného kurzu nebo načítejte dynamicky
# ==============================================================================
EXCHANGE_RATE_EUR_CZK = 25.0

# ==============================================================================
# Funkce pro převod textového řetězce geometrie (souřadnicový řetězec) na shapely Polygon
# ==============================================================================
def parse_geometry(geom_str):
    """
    Předpokládá, že geom_str obsahuje souřadnice oddělené mezerou, např.:
      "x1 y1 x2 y2 ... xN yN"
    Vrací objekt shapely.geometry.Polygon.
    """
    try:
        coords = list(map(float, geom_str.split()))
        points = [(coords[i], coords[i+1]) for i in range(0, len(coords), 2)]
        return Polygon(points)
    except Exception as e:
        logger.error(f"Chyba při převodu geometrie: {e}")
        return None

# ==============================================================================
# Připojení k databázi VALUO pomocí SQLAlchemy
# ==============================================================================
params_conn = urllib.parse.quote_plus(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=localhost;"
    "Database=VALUO;"
    "Trusted_Connection=yes;"
)
connection_url = f"mssql+pyodbc:///?odbc_connect={params_conn}"
engine = create_engine(connection_url)

# ==============================================================================
# Pomocné funkce pro formátování
# ==============================================================================
def format_thousand_space(value):
    """
    Převede číselnou hodnotu na řetězec bez desetinných míst
    a s oddělením tisíců mezerou (např. 123456.789 -> '123 457').
    """
    try:
        return f"{float(value):,.0f}".replace(",", " ")
    except (ValueError, TypeError):
        return str(value)

def format_datum_podani(datum_str):
    """
    Převede datum 'YYYY-MM-DD HH:MM:SS.sss' na formát:
    MĚSÍC/ROK, (původní celý datum/časový údaj).
    Např. '2024-12-02 13:17:27.000' -> '<strong>12/2024</strong>, (2024-12-02 13:17:27.000)'
    Pokud parsování selže, vrátí původní řetězec.
    """
    try:
        dt_parsed = pd.to_datetime(datum_str)
        month_year = dt_parsed.strftime("%m/%Y")
        return f"<strong>{month_year}</strong>, ({datum_str})"
    except:
        return datum_str

# ==============================================================================
# --- Agregace tooltipu pro POZEMKY ---
# ==============================================================================
def aggregate_tooltip_text_formatted_pozemky(group):
    # Vybereme pouze první záznam pro každé unikátní cislo_vkladu
    unique_entries = group.drop_duplicates(subset=['cislo_vkladu'])
    tooltip_entries = []
    jc_values = []  # Pro výpočet průměrné JC

    # Procházíme maximálně 4 záznamy
    for idx, row in unique_entries.head(4).iterrows():
        datum_podani_val = format_datum_podani(row['datum_podani'])
        plocha_val = format_thousand_space(row['plocha'])
        try:
            cu_original = float(row['cenovy_udaj'])
        except (ValueError, TypeError):
            cu_original = 0.0
        try:
            jc_original = float(row['JC'])
        except (ValueError, TypeError):
            jc_original = 0.0

        mena = row['mena']
        if mena == 'EUR':
            cu_czk = cu_original * EXCHANGE_RATE_EUR_CZK
            jc_czk = jc_original * EXCHANGE_RATE_EUR_CZK
            cenovy_udaj_val = f"{format_thousand_space(cu_original)} EUR / {format_thousand_space(cu_czk)} CZK"
            jc_val_str = f"{format_thousand_space(jc_czk)} CZK/m²"
        else:
            cu_czk = cu_original
            jc_czk = jc_original
            cenovy_udaj_val = f"{format_thousand_space(cu_czk)} CZK"
            jc_val_str = f"{format_thousand_space(jc_czk)} CZK/m²"

        jc_values.append(jc_czk)

        entry = (
            f"<table style='font-size:10px;'>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Číslo vkladu:</td>"
            f"<td style='padding-left: 10px;'>{row['cislo_vkladu']}</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Počet parcel v rámci vkladu:</td>"
            f"<td style='padding-left: 10px;'>{row['#PARCEL']}</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Datum podání:</td>"
            f"<td style='padding-left: 10px;'>{datum_podani_val}</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Okres:</td>"
            f"<td style='padding-left: 10px;'>{row['okres']}</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Katastrální území:</td>"
            f"<td style='padding-left: 10px;'>{row['kat_uzemi']}</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Kód kat. území:</td>"
            f"<td style='padding-left: 10px;'>{row['upper_zoning_id']}</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Typ:</td>"
            f"<td style='padding-left: 10px;'>{row['typ']}</td></tr>"            
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Parc.č.:</td>"
            f"<td style='padding-left: 10px;'>{row['parcel_number']}</td></tr>"            
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Plocha:</td>"
            f"<td style='padding-left: 10px;'>{plocha_val} m²</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Cenový údaj:</td>"
            f"<td style='padding-left: 10px;'>{cenovy_udaj_val}</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>JC:</td>"
            f"<td style='padding-left: 10px;'>{jc_val_str}</td></tr>"
            f"</table>"
        )
        tooltip_entries.append(entry)

    aggregated_text = "<hr>".join(tooltip_entries)

    # Přidáme výpočet průměrné JC
    if len(jc_values) > 0:
        avg_jc_czk = sum(jc_values) / len(jc_values)
        avg_jc_str = format_thousand_space(avg_jc_czk)
        aggregated_text += (
            f"<br><table style='font-size:14px;'><tr>"
            f"<td style='font-weight:bold; vertical-align: top;'>Průměrná JC:</td>"
            f"<td style='padding-left: 10px;'>{avg_jc_str} CZK/m²</td>"
            f"</tr></table>"
        )

    unique_count = unique_entries.shape[0]
    aggregated_text += f"<br><span style='font-size:12px; font-weight:bold;'>Celkem unikátních čísel vkladu: {unique_count}</span>"
    return aggregated_text

# ==============================================================================
# --- Agregace tooltipu pro BYTY ---
# ==============================================================================
def aggregate_tooltip_text_formatted_byty_for_parcel(parcel_group):
    lines_for_cislo_vkladu = []
    jc_values = []  # Pro výpočet průměrné JC
    computed = []  # Uložíme výsledky pro každé unikátní cislo_vkladu

    grouped_by_cislo = parcel_group.groupby('cislo_vkladu')
    for cislo_vkladu, subdf in grouped_by_cislo:
        plocha_avg = subdf['plocha'].astype(float).mean(skipna=True) or 0.0
        mena = subdf.iloc[0].get('mena', 'CZK')
        cenovy_udaj_avg_original = subdf['cenovy_udaj'].astype(float).mean(skipna=True) or 0.0

        if mena == 'EUR':
            cenovy_udaj_avg_czk = cenovy_udaj_avg_original * EXCHANGE_RATE_EUR_CZK
            cenovy_udaj_val_for_tooltip = (
                f"{format_thousand_space(cenovy_udaj_avg_original)} EUR / "
                f"{format_thousand_space(cenovy_udaj_avg_czk)} CZK"
            )
        else:
            cenovy_udaj_avg_czk = cenovy_udaj_avg_original
            cenovy_udaj_val_for_tooltip = f"{format_thousand_space(cenovy_udaj_avg_czk)} CZK"

        if plocha_avg == 0:
            jc_calc = 0.0
        else:
            jc_calc = cenovy_udaj_avg_czk / plocha_avg

        jc_values.append(jc_calc)
        computed.append((cislo_vkladu, jc_calc, subdf.iloc[0], cenovy_udaj_val_for_tooltip, plocha_avg))

    # Vypíšeme pouze první 4 segmenty
    for i, (cislo_vkladu, jc_calc, row0, cenovy_udaj_val_for_tooltip, plocha_avg) in enumerate(computed):
        if i >= 4:
            break
        datum_podani_val = format_datum_podani(row0['datum_podani'])
        plocha_avg_str = format_thousand_space(plocha_avg)
        jc_calc_str = f"{format_thousand_space(jc_calc)} CZK/m²"
        okres = row0.get('okres', '')
        kat_uzemi = row0.get('kat_uzemi', '')
        adresa = row0.get('adresa', '')
        typ = row0.get('typ', '')
        popis = row0.get('popis', '')
        line_html = (
            f"<table style='font-size:10px;'>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Číslo vkladu:</td>"
            f"<td style='padding-left: 10px;'>{cislo_vkladu}</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Datum podání:</td>"
            f"<td style='padding-left: 10px;'>{datum_podani_val}</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Okres:</td>"
            f"<td style='padding-left: 10px;'>{okres}</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Katastrální území:</td>"
            f"<td style='padding-left: 10px;'>{kat_uzemi}</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Adresa:</td>"
            f"<td style='padding-left: 10px;'>{adresa}</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Typ:</td>"
            f"<td style='padding-left: 10px;'>{typ}</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Plocha (prům.):</td>"
            f"<td style='padding-left: 10px;'>{plocha_avg_str} m²</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Cenový údaj (prům.):</td>"
            f"<td style='padding-left: 10px;'>{cenovy_udaj_val_for_tooltip}</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>JC:</td>"
            f"<td style='padding-left: 10px;'>{jc_calc_str}</td></tr>"
            f"<tr><td style='font-weight:bold; vertical-align: top;'>Popis:</td>"
            f"<td style='padding-left: 10px; max-width:150px; word-wrap: break-word; white-space: normal;'>{popis}</td></tr>"
            f"</table>"
        )
        lines_for_cislo_vkladu.append(line_html)

    result_html = "<hr>".join(lines_for_cislo_vkladu)

    # Reset indexu kvůli dostupnosti 'parcel_number'
    parcel_group = parcel_group.reset_index()

    # Výpočet průměrné JC
    if len(jc_values) > 0:
        avg_jc_for_parcel = sum(jc_values) / len(jc_values)
        avg_jc_str = format_thousand_space(avg_jc_for_parcel)
        result_html += (
            f"<br><table style='font-size:14px;'><tr>"
            f"<td style='font-weight:bold; vertical-align: top;'>Průměrná JC (v rámci budovy / parcely):</td>"
            f"<td style='padding-left: 10px;'>{avg_jc_str} CZK/m²</td>"
            f"</tr></table>"
        )
    total_unique = len(computed)
    result_html += f"<br><span style='font-size:12px; font-weight:bold;'>Celkem unikátních čísel vkladu: {total_unique}</span>"
    return pd.Series({
        'tooltip_text': result_html,
        'avg_jc': avg_jc_for_parcel if len(jc_values) > 0 else None
})


# ==============================================================================
# ---------------------- Data POZEMKY: Dotaz + úprava + vykreslení -------------
# ==============================================================================
df_pozemky = pd.read_sql(query_pozemky, engine)
logger.info("Načtených záznamů z KN_parcel_data:")
logger.info(df_pozemky.head())

df_pozemky = df_pozemky.loc[:, ~df_pozemky.columns.duplicated()]
df_pozemky['geom'] = df_pozemky['geometry'].apply(parse_geometry)

gdf_pozemky = gpd.GeoDataFrame(df_pozemky, geometry='geom', crs="EPSG:5514")
gdf_pozemky = gdf_pozemky.drop(columns=['geometry']).rename(columns={'geom': 'geometry'})
gdf_pozemky = gdf_pozemky.set_geometry("geometry").to_crs(epsg=4326)

tooltip_aggregated_poz = (
    df_pozemky
    .groupby('parcel_number')
    .apply(aggregate_tooltip_text_formatted_pozemky)
    .reset_index()
)
tooltip_aggregated_poz.columns = ['parcel_number', 'tooltip_text']

gdf_poz_agg = gdf_pozemky.dissolve(by='parcel_number', as_index=False)
gdf_poz_agg = gdf_poz_agg.merge(tooltip_aggregated_poz, on='parcel_number', how='left')

def get_color_pozemky(jc_value):
    if jc_value <= 299:
        return "#006400"   # tmavě zelená
    elif jc_value <= 999:
        return "red"
    elif jc_value <= 4999:
        return "yellow"
    elif jc_value <= 9999:
        return "orange"
    elif jc_value <= 19999:
        return "lightblue"
    elif jc_value <= 49999:
        return "darkblue"
    else:
        return "purple"

def style_function_pozemky(feature):
    try:
        jc_value = float(feature["properties"]["JC"])
    except (KeyError, TypeError, ValueError):
        jc_value = 0
    return {
        'color': 'black',
        'weight': 2,
        'fillColor': get_color_pozemky(jc_value),
        'fillOpacity': 0.5,
    }

tooltip_pozemky = folium.GeoJsonTooltip(
    fields=['tooltip_text'],
    aliases=[''],
    localize=True,
    sticky=True,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 3px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=300,
)

gdf_poz_agg_json = gdf_poz_agg.to_json(default=str)

# ==============================================================================
# Funkce pro přidání textových popisků (labelů) na každou parcelu
# ==============================================================================
def add_jc_labels(map_obj, geodf, jc_value):
    """
    Přidá textové popisky na mapu, které zobrazují hodnotu JC (předanou jako argument)
    ve formátu "JC = value CZK/m²" na každý centroid geometrie.
    """
    try:
        jc_value = float(jc_value)
        jc_label = f"{jc_value:,.0f} CZK/m²"
    except (ValueError, TypeError):
        jc_label = "JC = N/A"

    for idx, row in geodf.iterrows():
        if row.geometry is None:
            continue
        centroid = row.geometry.centroid

        folium.map.Marker(
            [centroid.y, centroid.x],
            icon=folium.DivIcon(
                html=f"""
                <div style="font-size:10pt; font-weight:bold; color:black; background-color:rgba(255,255,255,0.0);
                            border:0px solid gray; border-radius:5px; padding:5px; white-space: nowrap;">
                    {jc_label}
                </div>
                """
            )
        ).add_to(map_obj)



# ==============================================================================
# ---------------------- Data BYTY: Dotaz + úprava + vykreslení ----------------
# ==============================================================================
df_byty = pd.read_sql(query_byty, engine)
logger.info("Načtených záznamů z KN_bytu_data:")
logger.info(df_byty.head())

df_byty = df_byty.loc[:, ~df_byty.columns.duplicated()]
df_byty['geom'] = df_byty['geometry'].apply(parse_geometry)

gdf_byty = gpd.GeoDataFrame(df_byty, geometry='geom', crs="EPSG:5514")
gdf_byty = gdf_byty.drop(columns=['geometry']).rename(columns={'geom': 'geometry'})
gdf_byty = gdf_byty.set_geometry("geometry").to_crs(epsg=4326)

tooltip_aggregated_byty = (
    df_byty
    .groupby('parcel_number')
    .apply(aggregate_tooltip_text_formatted_byty_for_parcel)
    .reset_index()
)
tooltip_aggregated_byty.columns = ['parcel_number', 'tooltip_text', 'avg_jc']


gdf_byty_agg = gdf_byty.dissolve(by='parcel_number', as_index=False)
gdf_byty_agg = gdf_byty_agg.merge(tooltip_aggregated_byty, on='parcel_number', how='left')



def get_color_byty(jc_value):
    if jc_value <= 69999:
        return "green"
    elif jc_value <= 89999:
        return "yellow"
    elif jc_value <= 99999:
        return "orange"
    elif jc_value <= 119999:
        return "red"  
    elif jc_value <= 149999:
        return "purple"
    else:
        return "darkblue"  # dark red in hexadecimal format

def style_function_byty(feature):
    try:
        jc_value = float(feature["properties"]["JC"])
    except (KeyError, TypeError, ValueError):
        jc_value = 0
    return {
        'color': 'black',
        'weight': 2,
        'fillColor': get_color_byty(jc_value),
        'fillOpacity': 0.5,
    }

tooltip_byty = folium.GeoJsonTooltip(
    fields=['tooltip_text'],
    aliases=[''],
    localize=True,
    sticky=True,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 3px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=300,
)

gdf_byty_agg_json = gdf_byty_agg.to_json(default=str)

# ==============================================================================
# ---------------------- Vytvoření interaktivní mapy ---------------------------
# ==============================================================================
center_lat = gdf_poz_agg.geometry.centroid.y.mean()
center_lon = gdf_poz_agg.geometry.centroid.x.mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=15)

# Získáme popisky JC z tooltip_text nebo (lépe) předem spočítané JC hodnoty
for idx, row in gdf_byty_agg.iterrows():
    # můžeš tam mít sloupec JC nebo ho získat jinak
    jc_label = row['avg_jc']  
    add_jc_labels(m, gdf_byty_agg.iloc[[idx]], jc_label)

# Vrstva BYTY
folium.GeoJson(
    gdf_byty_agg_json,
    name="BYTY",
    style_function=style_function_byty,
    tooltip=tooltip_byty
).add_to(m)

# Vrstva POZEMKY
folium.GeoJson(
    gdf_poz_agg_json,
    name="POZEMKY",
    style_function=style_function_pozemky,
    tooltip=tooltip_pozemky
).add_to(m)


# Přidání popisků do mapy
#add_jc_labels(m, gdf_poz_agg)  # Přidání popisků pro pozemky
#add_jc_labels(m, gdf_byty_agg)  # Přidání popisků pro byty

# Ortofoto mapa
folium.TileLayer(
    tiles='http://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Esri World Imagery',
    name='Ortofoto Mapa',
    overlay=True,
    control=True
).add_to(m)

# Dynamická vrstva Územního plánu
class DynamicArcGISTileLayer(MacroElement):
    _template = Template(u"""
        {% macro script(this, kwargs) %}
            var DynamicLayer = L.TileLayer.extend({
                getTileUrl: function(coords) {
                    var tileSize = 256;
                    var initialResolution = 2 * Math.PI * 6378137 / tileSize;
                    var originShift = 2 * Math.PI * 6378137 / 2.0;
                    var resolution = initialResolution / Math.pow(2, coords.z);
                    var minx = coords.x * tileSize * resolution - originShift;
                    var maxx = (coords.x + 1) * tileSize * resolution - originShift;
                    var miny = originShift - (coords.y + 1) * tileSize * resolution;
                    var maxy = originShift - coords.y * tileSize * resolution;
                    var bbox = [minx, miny, maxx, maxy].join(",");
                    return "{{ this.url }}?bbox=" + bbox +
                           "&bboxSR=102100&imageSR=102100&size=256,256" +
                           "&format=png32&transparent=true&layers=show:0&f=image";
                }
            });
            var dynamicLayer = new DynamicLayer();
            dynamicLayer.addTo({{ this._parent.get_name() }});
        {% endmacro %}
    """)
    def __init__(self, url):
        super(DynamicArcGISTileLayer, self).__init__()
        self._name = 'DynamicArcGISTileLayer'
        self.url = url

arcgis_export_url = "https://gs-pub.praha.eu/arcgis/rest/services/pup/uzemni_plan_platny/MapServer/export"
dynamic_group = folium.FeatureGroup(
    name="Územní plán Hlavního města Prahy (výkres č. 4) – Plán využití ploch",
    overlay=True,
    control=True
)
dynamic_layer = DynamicArcGISTileLayer(arcgis_export_url)
dynamic_group.add_child(dynamic_layer)
m.add_child(dynamic_group)

folium.LayerControl().add_to(m)

output_file = "DATA_POZEMKY_BYTY_JC.html"
m.save(output_file)
logger.info(f"Interaktivní mapa byla uložena do souboru '{output_file}'.")
webbrowser.open(output_file)


2025-05-28 08:55:22,138 - INFO - Načtených záznamů z KN_parcel_data:
2025-05-28 08:55:22,138 - INFO - Načtených záznamů z KN_parcel_data:
2025-05-28 08:55:22,138 - INFO - Načtených záznamů z KN_parcel_data:
2025-05-28 08:55:22,138 - INFO - Načtených záznamů z KN_parcel_data:
2025-05-28 08:55:22,141 - INFO -       id           timestamp      cislo_vkladu        datum_podani  \
0  18218 2025-02-03 19:18:12     V-10/2023-101 2023-01-02 08:58:57   
1   9248 2025-02-03 16:28:22  V-10027/2023-211 2023-12-21 11:54:17   
2   9249 2025-02-03 16:28:22  V-10027/2023-211 2023-12-21 11:54:17   
3   9250 2025-02-03 16:28:22  V-10027/2023-211 2023-12-21 11:54:17   
4  40903 2025-05-28 08:29:54  V-10091/2024-209 2024-08-21 08:30:00   

      datum_zplatneni        listina nemovitost                   typ  \
0 2023-01-24 13:58:23  Smlouva kupní    parcela           jiná plocha   
1 2024-01-25 12:28:43  Smlouva kupní    parcela             orná půda   
2 2024-01-25 12:28:43  Smlouva kupní    parcela  tr

True